In [1]:
# import dependencies
import win32com.client as wc
import xml.etree.ElementTree as ET
import pandas as pd
import mysql.connector as mc
import datetime as dt
from datetime import datetime
import pyodbc
from pdfrw import PdfReader, PdfWriter
import pdfrw 
import os
from pdf2image import convert_from_path
import img2pdf
from PIL import Image
import smtplib
import ssl
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import sys
import shutil
import pdfkit
import numpy as np
from bs4 import BeautifulSoup as Soup

# Get db credentials
from configTest import mysql_host, mysql_u, mysql_pw, vgc_host, vgc_u, vgc_pw, svr, db, sql_u, sql_pw, smtp_host, e_user, e_pw, port

In [2]:
# Show all columns in DFs
pd.set_option("display.max_columns", None)

In [3]:
# create batch_id
now = datetime.now()
batch_id = now.strftime("%Y%m%d%H%M%S")

In [4]:
# MySQL
def mysql_q (u, p, h, db, sql, cols, commit):
    # cols (0 = no, 1 = yes)
    # commit (select = 0, insert/update = 1)

    # connect to claim_qb_payments db
    cnx = mc.connect(user=u, password=p,
                    host=h,
                    database=db)
    cursor = cnx.cursor()

    # commit?
    if commit == 1:
        cursor.execute(sql)
        cnx.commit()
        sql_result = 0     
    else:
        cursor.execute(sql)
        sql_result = cursor.fetchall()

    # columns ?
    if cols == 1:
        columns=list([x[0] for x in cursor.description])
        # close connection
        cursor.close()
        cnx.close()
        # return query result [0] and columns [1]
        return sql_result
    else:
        # close connection
        cursor.close()
        cnx.close()
        # return query result
        return sql_result      

In [5]:
# PDF Concatenation Function
def ConCat_pdf (file_list, outfn):
    letter_path = './letters/staging/'
    writer = PdfWriter()
    for inputfn in file_list:
        writer.addpages(PdfReader(letter_path + inputfn).pages)

    outfile = outfn + '.pdf'
    fnNum = 0

    while (os.path.isfile(outfile) == True):
        fnNum += 1
        outfile = outfn + '-' + str(fnNum) + '.pdf'

    writer.write(outfile)
    return outfile

In [6]:
# Delete Function
def delete_file(del_file_path):
    if os.path.exists(del_file_path):
        os.remove(del_file_path)
    else: print (f"{del_file_path} does not exist")

In [7]:
# Create PDF Function
def fill_pdf(input_pdf_path, output_pdf_path, data_dict):
    ANNOT_KEY = '/Annots'
    ANNOT_FIELD_KEY = '/T'
    ANNOT_VAL_KEY = '/V'
    ANNOT_RECT_KEY = '/Rect'
    SUBTYPE_KEY = '/Subtype'
    WIDGET_SUBTYPE_KEY = '/Widget'

    template_pdf = pdfrw.PdfReader(input_pdf_path)
    
    for page in template_pdf.pages:
        annotations = page[ANNOT_KEY]
        for annotation in annotations:
            if annotation[SUBTYPE_KEY] == WIDGET_SUBTYPE_KEY:
                if annotation[ANNOT_FIELD_KEY]:
                    key = annotation[ANNOT_FIELD_KEY][1:-1]
                    if key in data_dict.keys():
                        if type(data_dict[key]) == bool:
                            if data_dict[key] == True:
                                annotation.update(pdfrw.PdfDict(
                                    AS=pdfrw.PdfName('Yes')))
                        else:
                            annotation.update(
                                pdfrw.PdfDict(V='{}'.format(data_dict[key]))
                            )
                            annotation.update(pdfrw.PdfDict(AP=''))
    template_pdf.Root.AcroForm.update(pdfrw.PdfDict(NeedAppearances=pdfrw.PdfObject('true')))

    pdfrw.PdfWriter().write(output_pdf_path, template_pdf)

    # return 1

In [8]:
# Flatten PDF function
def flatten_pdf(flat_output, img_file):
    # Fillable PDF to Image
    images = convert_from_path(flat_output, dpi=300, size=(2550,3300))
    for i in range(len(images)):
   
    # Save pages as images in the pdf
        images[i].save(img_file + '.png', 'PNG')
    
    # Delete Fillable PDF
    delete_file(img_file + '.pdf')
    
    # opening image
    image_file = Image.open(img_file + '.png')
    
    # Image to Flat PDF
    # define paper size
    letter = (img2pdf.in_to_pt(8.5), img2pdf.in_to_pt(11))
    layout = img2pdf.get_layout_fun(letter)
    # converting into chunks using img2pdf
    pdf_bytes = img2pdf.convert(image_file.filename, layout_fun=layout)
    
    # opening or creating pdf file
    flat_pdf = f"{img_file}.pdf"
    file = open(flat_pdf, "wb")
    
    # writing pdf files with chunks
    file.write(pdf_bytes)
    
    # Add file name to file_name list
    # file_list.append(flat_pdf)

    # closing image file
    image_file.close()
    
     # Delete Fillable PDF
    delete_file(img_file + '.png')

    # closing pdf file
    file.close()

    # return 1

In [9]:
# GAP Letter function
def gap_letter(template_df, pdf_template, position):
    gap_path = './letters/staging/'

    template_df['payment_amount'] = template_df['payment_amount'].map('${:,.2f}'.format)
    template_df['loss_date'] = pd.to_datetime(template_df['loss_date']).dt.strftime('%B %d, %Y')
    template_df['StateDesc'] = template_df['StateDesc'].astype(str).replace({'None':''})
    template_df['StateCode'] = template_df['StateCode'].astype(str).replace({'None':''})
    template_df['f_lang'] = template_df['f_lang'].astype(str).replace({'None':''})
    letter_date = f"{datetime.now():%B %d, %Y}"   

    for index, row in template_df.iterrows():

        # empty dict
        data_dict = {}
        # store field data in dictionary
        data_dict = {
            'Date': letter_date,
            'Lender': template_df.loc[index]['alt_name'],
            'Contact': template_df.loc[index]['contact'],
            'Address': template_df.loc[index]['address1'],
            'City_St_Zip': f"{template_df.loc[index]['city']}, {template_df.loc[index]['state']} {template_df.loc[index]['zip']}",
            'Lender2': template_df.loc[index]['alt_name'],
            'Borrower': f"{template_df.loc[index]['first']} {template_df.loc[index]['last']}",
            'Claim_Nbr': template_df.loc[index]['claim_nbr'],
            'Acct_Nbr': template_df.loc[index]['acct_number'],
            'DOL': template_df.loc[index]['loss_date'],
            'GAP_Amt': template_df.loc[index]['payment_amount'],
            'State': template_df.loc[index]['StateDesc'],
            'St_Code': template_df.loc[index]['StateCode'],
            'Fraud': template_df.loc[index]['f_lang'],
        }

        # store paths as variables
        output_file = f"{template_df.loc[index]['claim_nbr']}-{position}.pdf"
        output_path_fn = f"{gap_path}{output_file}"

        fill_pdf(pdf_template, output_path_fn, data_dict)

        # Set File Paths
        flat_output = f"{os.path.dirname(os.path.abspath(output_file))}\{gap_path}\{output_file}"
        img_file = f"{os.path.dirname(os.path.abspath(output_file))}\{gap_path}\{template_df.loc[index]['claim_nbr']}-{position}"

        # Flatten pdf using flatten_pdf function
        flatten_pdf(flat_output, img_file) 

    # return 1

In [10]:
# GAP Calculation function
def calculations(template_df, pdf_template, position):
    gap_path = './letters/staging/'

    template_df['loss_date'] = pd.to_datetime(template_df['loss_date']).dt.strftime('%B %d, %Y')
    template_df['last_payment'] = pd.to_datetime(template_df['last_payment']).dt.strftime('%B %d, %Y')  
    template_df['incp_date'] = pd.to_datetime(template_df['incp_date']).dt.strftime('%B %d, %Y')    

    template_df['payoff'] = template_df['payoff'].map('${:,.2f}'.format)
    template_df['past_due'] = template_df['past_due'].map('${:,.2f}'.format)
    template_df['late_fees'] = template_df['late_fees'].map('${:,.2f}'.format)
    template_df['skip_pymts'] = template_df['skip_pymts'].map('${:,.2f}'.format)
    template_df['skip_fees'] = template_df['skip_fees'].map('${:,.2f}'.format)
    template_df['primary_pymt'] = template_df['primary_pymt'].map('${:,.2f}'.format)
    template_df['excess_deductible'] = template_df['excess_deductible'].map('${:,.2f}'.format)
    template_df['scr'] = template_df['scr'].map('${:,.2f}'.format)
    template_df['clr'] = template_df['clr'].map('${:,.2f}'.format)
    template_df['cdr'] = template_df['cdr'].map('${:,.2f}'.format)
    template_df['oref'] = template_df['oref'].map('${:,.2f}'.format)
    template_df['salvage'] = template_df['salvage'].map('${:,.2f}'.format)
    template_df['prior_dmg'] = template_df['prior_dmg'].map('${:,.2f}'.format)
    template_df['over_ltv'] = template_df['over_ltv'].map('${:,.2f}'.format)
    template_df['other1_amt'] = template_df['other1_amt'].map('${:,.2f}'.format)
    template_df['other2_amt'] = template_df['other2_amt'].map('${:,.2f}'.format)
    template_df['gap_payable'] = template_df['gap_payable'].map('${:,.2f}'.format)
    template_df['balance_last_pay'] = template_df['balance_last_pay'].map('${:,.2f}'.format)
    template_df['per_day'] = template_df['per_day'].map('${:,.2f}'.format)
    template_df['deductible'] = template_df['deductible'].map('${:,.2f}'.format)
    template_df['subtotal'] = template_df['subtotal'].map('${:,.2f}'.format)
    template_df['nbr_of_days'] = template_df['nbr_of_days'].map('{:,.0f}'.format)
    template_df['interest_rate'] = template_df['interest_rate'].map('{:,.2f}%'.format)
    template_df['ltv'] = template_df['ltv'].map('{:,.2f}%'.format)
    template_df['ltv_limit'] = template_df['ltv_limit'].map('{:,.2f}%'.format)
    template_df['percent_uncovered'] = template_df['percent_uncovered'].map('{:,.2f}%'.format)
    template_df['covered_fin_amount'] = template_df['covered_fin_amount'].map('${:,.2f}'.format)
    template_df['Amt_Fin'] = template_df['Amt_Fin'].map('${:,.2f}'.format)
    template_df['nada_value'] = template_df['nada_value'].map('${:,.2f}'.format)

    for index, row in template_df.iterrows():

        # empty dict
        data_dict = {}
        # store field data in dictionary
        data_dict = {
            'Claim_Number': template_df.loc[index]['claim_nbr'],
            'Status': 'Paid',
            'Borrower': f"{template_df.loc[index]['first']} {template_df.loc[index]['last']}",
            'Vehicle': template_df.loc[index]['vehicle'],
            'Date_Of_Loss': template_df.loc[index]['loss_date'],
            'Type_Of_Loss': template_df.loc[index]['loss_type'],
            'Lender': template_df.loc[index]['alt_name'],         
            'Lender_Contact': template_df.loc[index]['contact'],
            'Insurance_Carrier': template_df.loc[index]['carrier'],
            'Inception_Date': template_df.loc[index]['incp_date'],
            'Deductible': template_df.loc[index]['deductible'],
            'Payoff': template_df.loc[index]['payoff'],
            'Past_Due': template_df.loc[index]['past_due'],
            'Late_Fees': template_df.loc[index]['late_fees'],
            'Skips': template_df.loc[index]['skip_pymts'],
            'Skip_Fees': template_df.loc[index]['skip_fees'],
            'Primary': template_df.loc[index]['primary_pymt'],
            'Deductible_Excess': template_df.loc[index]['excess_deductible'],
            'SCR': template_df.loc[index]['scr'],
            'CL_Refund': template_df.loc[index]['clr'],
            'CD_Refund': template_df.loc[index]['cdr'],
            'O_Refund': template_df.loc[index]['oref'],
            'Salvage': template_df.loc[index]['salvage'],
            'Prior_Damage': template_df.loc[index]['prior_dmg'],
            'Over_LTV': template_df.loc[index]['over_ltv'],
            'Other1_Description': template_df.loc[index]['other1_description'],
            'Other2_Description': template_df.loc[index]['other2_description'],
            'Other1': template_df.loc[index]['other1_amt'],
            'Other2': template_df.loc[index]['other2_amt'],
            'Deduction_Subtotal': template_df.loc[index]['subtotal'],
            'GAP_Amt': template_df.loc[index]['gap_payable'], 
            'Last_pymt_date': template_df.loc[index]['last_payment'], 
            'DOL': template_df.loc[index]['loss_date'],
            'Number_of_days': template_df.loc[index]['nbr_of_days'], 
            'Loan_Payoff_As_of_DOL': template_df.loc[index]['payoff'],
            'Bal_as_of_last_pymt': template_df.loc[index]['balance_last_pay'],
            'Interest_Rate': template_df.loc[index]['interest_rate'],
            'Interest_Per_Day': template_df.loc[index]['per_day'],
            'Amt_financed': template_df.loc[index]['Amt_Fin'],
            'ACV': template_df.loc[index]['nada_value'], 
            'LTV': template_df.loc[index]['ltv'],
            'Max_Amt_Financed': template_df.loc[index]['covered_fin_amount'],
            'LTV_limit': template_df.loc[index]['ltv_limit'],
            'Percentage_Not_Covered': template_df.loc[index]['percent_uncovered']
        }

        # store paths as variables
        output_file = f"{template_df.loc[index]['claim_nbr']}-{position}.pdf"
        output_path_fn = f"{gap_path}{output_file}"

        fill_pdf(pdf_template, output_path_fn, data_dict)

        # Set File Paths
        flat_output = f"{os.path.dirname(os.path.abspath(output_file))}\{gap_path}\{output_file}"
        img_file = f"{os.path.dirname(os.path.abspath(output_file))}\{gap_path}\{template_df.loc[index]['claim_nbr']}-{position}"

        # Flatten pdf using flatten_pdf function
        flatten_pdf(flat_output, img_file)

In [11]:
def clear_dir (path, ext):
    for x in os.listdir(path):
        if x.endswith(ext):
            os.remove(os.path.join(path, x))

In [12]:
def fileList (path, ext):
    file_list = []
    for x in os.listdir(path):
        if x.endswith(ext):
            file_list.append(x)

    # sort list to collate pages
    file_list.sort()
    
    return file_list

In [13]:
# TotalRestart Calculation function
def tr_calculations(template_df, pdf_template, position):
    gap_path = './letters/staging/'

    template_df['loss_date'] = pd.to_datetime(template_df['loss_date']).dt.strftime('%B %d, %Y')
    template_df['incp_date'] = pd.to_datetime(template_df['incp_date']).dt.strftime('%B %d, %Y')

    template_df['primary_pymt'] = template_df['primary_pymt'].map('${:,.2f}'.format)
    template_df['excess_deductible'] = template_df['excess_deductible'].map('${:,.2f}'.format)
    template_df['scr'] = template_df['scr'].map('${:,.2f}'.format)
    template_df['clr'] = template_df['clr'].map('${:,.2f}'.format)
    template_df['cdr'] = template_df['cdr'].map('${:,.2f}'.format)
    template_df['oref'] = template_df['oref'].map('${:,.2f}'.format)
    template_df['salvage'] = template_df['salvage'].map('${:,.2f}'.format)
    template_df['prior_dmg'] = template_df['prior_dmg'].map('${:,.2f}'.format)
    template_df['other1_amt'] = template_df['other1_amt'].map('${:,.2f}'.format)
    template_df['other2_amt'] = template_df['other2_amt'].map('${:,.2f}'.format)
    template_df['other3_amt'] = template_df['other3_amt'].map('${:,.2f}'.format)    
    template_df['gap_payable'] = template_df['gap_payable'].map('${:,.2f}'.format)
    template_df['subtotal'] = template_df['subtotal'].map('${:,.2f}'.format)
    template_df['nada_value'] = template_df['nada_value'].map('${:,.2f}'.format)
    template_df['max_benefit'] = template_df['max_benefit'].map('${:,.2f}'.format)    
    template_df['totalrestart_payable'] = template_df['totalrestart_payable'].map('${:,.2f}'.format)

    for index, row in template_df.iterrows():

        # empty dict
        data_dict = {}
        # store field data in dictionary
        data_dict = {
            'Claim_Number': template_df.loc[index]['claim_nbr'],
            'Status': 'Paid',
            'Borrower': f"{template_df.loc[index]['first']} {template_df.loc[index]['last']}",
            'Vehicle': template_df.loc[index]['vehicle'],
            'Date_Of_Loss': template_df.loc[index]['loss_date'],
            'Type_Of_Loss': template_df.loc[index]['loss_type'],
            'Lender': template_df.loc[index]['alt_name'],         
            'Lender_Contact': template_df.loc[index]['contact'],
            'Inception_Date': template_df.loc[index]['incp_date'],
            'Max_Potential_Benefit': template_df.loc[index]['max_benefit'],
            'Membership_Term': template_df.loc[index]['term'],
            'Primary': template_df.loc[index]['primary_pymt'],
            'Deductible_Excess': template_df.loc[index]['excess_deductible'],
            'SCR': template_df.loc[index]['scr'],
            'CL_Refund': template_df.loc[index]['clr'],
            'CD_Refund': template_df.loc[index]['cdr'],
            'O_Refund': template_df.loc[index]['oref'],
            'Salvage': template_df.loc[index]['salvage'],
            'Prior_Damage': template_df.loc[index]['prior_dmg'],
            'Other1_Description': template_df.loc[index]['other1_description'],
            'Other2_Description': template_df.loc[index]['other2_description'],
            'Other3_Description': template_df.loc[index]['other3_description'],
            'Other1': template_df.loc[index]['other1_amt'],
            'Other2': template_df.loc[index]['other2_amt'],
            'Other3': template_df.loc[index]['other3_amt'],
            'TR_Deduction_Subtotal': template_df.loc[index]['subtotal'],
            'GAP_Amt': template_df.loc[index]['gap_payable'], 
            'ACV': template_df.loc[index]['nada_value'], 
            'TR_Amt': template_df.loc[index]['totalrestart_payable']
        }

        # store paths as variables
        output_file = f"{template_df.loc[index]['claim_nbr']}-{position}.pdf"
        output_path_fn = f"{gap_path}{output_file}"

        fill_pdf(pdf_template, output_path_fn, data_dict)

        # Set File Paths
        flat_output = f"{os.path.dirname(os.path.abspath(output_file))}\{gap_path}\{output_file}"
        img_file = f"{os.path.dirname(os.path.abspath(output_file))}\{gap_path}\{template_df.loc[index]['claim_nbr']}-{position}"

        # Flatten pdf using flatten_pdf function
        flatten_pdf(flat_output, img_file)

In [14]:
# Email
def send_email(toEmail, subject, msg_html, attachPath, *args):
    fromEmail = 'claims@visualgap.com'
    # attachPath = './Claim_Payments/letters/attachments/'

    # address message
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = fromEmail
    msg['To'] = ','.join(toEmail)

    # create body
    body_html = MIMEText(msg_html, 'html')
    msg.attach(body_html) 

    for arg in args:
        fName = attachPath + arg
        filename = os.path.abspath(fName)

        with open(filename, 'rb') as fn:
            attachment = MIMEApplication(fn.read())
            attachment.add_header('Content-Disposition', 'attachment', filename=arg)
            msg.attach(attachment)

    context = ssl.create_default_context()
    try:
        server = smtplib.SMTP(smtp_host, port)
        # check connection
        server.ehlo()  
        # Secure the connection
        server.starttls(context=context)  
        # check connection
        server.ehlo()
        server.login(e_user, e_pw)
        # Send email
        server.sendmail(fromEmail, toEmail, msg.as_string())

    except Exception as e:
        # Print any error messages
        print(e)
    finally:
        server.quit()

In [15]:
def update_tovgc_1(df):
    # Update toVGC to 1

    # cnx = mc.connect(user=mysql_u, password=mysql_pw,
    #                 host=mysql_host,
    #                 database='claim_qb_payments')
    # cursor = cnx.cursor()

    if len(df) > 0:
        for index, row in df.iterrows():
            err_sql = '''
                    UPDATE ready_to_be_paid
                    SET toVGC = 1
                    WHERE rtbp_id = {rtbp_id};
                    '''.format(rtbp_id=row['rtbp_id'])
    #         # Run SQL
    #         cursor.execute(err_sql)
    #         cnx.commit()

    # # close mysql connection
    # cursor.close()
    # cnx.close()

            # connect to claim_qb_payments db
            mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', err_sql, 0, 1)


In [16]:
# connect to DB
# cnx = mc.connect(user=vgc_u, password=vgc_pw,
#                 host=vgc_host,
#                 database='visualgap_claims')
# cursor = cnx.cursor()

In [17]:
# sql query for GAP claims that are RTBP
sql_file = '''
            SELECT c.claim_id, c.claim_nbr, c.carrier_id, cl.alt_name, cl.dealer_securityId, cl.contact, cl.address1,
                cl.city, cl.state, cl.zip, cl.payment_method, cb.first, cb.last, 
                IF(sq.gap_amt_paid > 0, 2,1) AS pymt_type_id, 
                IF(sq.gap_amt_paid > 0, ROUND(cc.gap_payable - sq.gap_amt_paid,2), cc.gap_payable) AS gap_due,
                COALESCE(NULLIF(cb.acct_number,''),'0') AS acct_nbr, c.loss_date, cl.title, cl.email2
            FROM claims c
            INNER JOIN claim_lender cl
                USING (claim_id)
            INNER JOIN claim_borrower cb
                USING (claim_id)
            INNER JOIN claim_calculations cc
                USING (claim_id)
            INNER JOIN claim_status cs
                ON (c.status_id = cs.status_id)
            LEFT JOIN (SELECT cp.claim_id, SUM(cp.payment_amount) AS gap_amt_paid
                    FROM claim_payments cp
                    INNER JOIN (SELECT c.claim_id
                                FROM claims c
                                INNER JOIN claim_status cs
                                    ON (c.status_id = cs.status_id)
                                WHERE cs.status_desc_id = 8) rtbp_sq
                        USING (claim_id)
                    WHERE payment_category_id = 1
                    GROUP BY cp.claim_id) sq
                ON (c.claim_id = sq.claim_id)
            WHERE cs.status_desc_id = 8;
            '''

In [18]:
# # execute sql
# cursor.execute(sql_file)

In [19]:
# save query results as DF
df = pd.DataFrame(mysql_q(vgc_u, vgc_pw, vgc_host, 'visualgap_claims', sql_file, 0, 0))
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1466,202109172488,9,Charlotte Metro Credit Union,22260,CMCU Contact,8829 Chapel Square Dr (Jareds Office),Cincinnati,OH,45249,Check,Sam,Hunt,1,5302.64,0,2021-09-02,,jared@visualgap.com
1,1471,202110182493,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,ANDREW,FLYNN,1,0.00,0,2021-09-10,jediracer13@gmail.com,
2,1472,202110182494,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,TYSHAUN,GRANT,1,5138.04,250510182,2021-08-30,jediracer13@gmail.com,
3,1469,202110182491,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,TIFFANY,LEGO,1,2144.05,0,2021-08-28,jediracer13@gmail.com,
4,1473,202110182495,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,ALEX,MITCHELL,1,0.00,0,2021-06-25,jediracer13@gmail.com,
5,1475,202110192497,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,OCTAVIO,PARTIDA,1,653.87,187895-01,2021-08-28,jediracer13@gmail.com,
6,1470,202110182492,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,MARY,SCHEIBLE,1,0.00,6343056-50,2021-10-06,jediracer13@gmail.com,
7,1476,202110192498,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,SHERRY,VOGT,1,0.00,136324-02L,2021-07-27,jediracer13@gmail.com,
8,1478,202110192500,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,ASHLEY,MAPP,1,2820.16,22004112-030,2021-09-18,jediracer13@gmail.com,
9,1479,202110192501,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,LEELAND,WATKINS,1,103.35,804240-12,2021-08-18,jediracer13@gmail.com,


In [20]:
# # save query results as DF
# df = pd.DataFrame(cursor.fetchall())

In [21]:
# add column names
df_cols = ['claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'dealer_securityId', 'contact', 'address1', 'city', 'state', 'zip', 
                            'pymt_method', 'first', 'last', 'pymt_type_id', 'amount', 'acct_number', 'loss_date', 'email', 'email2']

df.columns = df_cols

In [22]:
# sql query for PLUS claims that are RTBP
sql_file_plus = '''
                SELECT sqp.claim_id, c.claim_nbr, c.carrier_id, cl.alt_name, cl.dealer_securityId, cl.contact, 
                    cl.address1, cl.city, cl.state, cl.zip, cl.payment_method, cb.first, cb.last, 
                    1 AS pymt_type_id, 
                    IF(cl.customer_securityId = 9401, 1500, 1000) AS gap_plus_due, COALESCE(NULLIF(cb.acct_number,''),'0') AS acct_nbr,
                    c.loss_date, cl.title, cl.email2
                FROM claims c
                INNER JOIN claim_lender cl
                    USING (claim_id)
                INNER JOIN claim_borrower cb
                    USING (claim_id)
                INNER JOIN (SELECT pb.claim_id
                            FROM claim_plus_benefit pb
                            WHERE status_desc_id = 8) sqp
                    USING (claim_id)
                INNER JOIN (SELECT c.claim_id
                            FROM claims c
                            INNER JOIN claim_status cs
                                ON (c.status_id = cs.status_id)  
                            WHERE cs.status_desc_id = 8
                                OR cs.status_desc_id = 4) sqg
                    USING (claim_id);
            '''

In [23]:
# # execute sql
# cursor.execute(sql_file_plus)

In [24]:
# # save query results as DF
# df2 = pd.DataFrame(cursor.fetchall())

In [25]:
df2 = pd.DataFrame(mysql_q(vgc_u, vgc_pw, vgc_host, 'visualgap_claims', sql_file_plus, 0, 0))
df2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1473,202110182495,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,ALEX,MITCHELL,1,1000,0,2021-06-25,jediracer13@gmail.com,
1,1475,202110192497,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,OCTAVIO,PARTIDA,1,1000,187895-01,2021-08-28,jediracer13@gmail.com,


In [26]:
# add column names
df2_cols = ['claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'dealer_securityId', 'contact', 'address1', 'city', 'state', 'zip', 
                            'pymt_method', 'first', 'last', 'pymt_type_id', 'amount', 'acct_number', 'loss_date', 'email', 'email2']

df2.columns = df2_cols

In [27]:
# sql query for TotalRestart claims that are RTBP
# manually entered carrier_id 12
sql_file_tr = '''
            SELECT sqp.claim_id, c.claim_nbr, 12 AS carrier_id, cl.alt_name, cl.dealer_securityId, cl.contact, cl.address1, 
            cl.city, cl.state, cl.zip, 'Check' AS payment_method, cb.first, cb.last, 1 AS pymt_type_id, 
            ctr.totalrestart_payable AS tr_due, COALESCE(NULLIF(cb.acct_number,''),'0') AS acct_nbr, c.loss_date, cl.title, cl.email2
            FROM claims c
            INNER JOIN claim_lender cl
                USING (claim_id)
            INNER JOIN claim_borrower cb
                USING (claim_id)
            INNER JOIN (SELECT pb.claim_id
                        FROM claim_totalrestart pb
                        WHERE status_desc_id = 8) sqp
                USING (claim_id)
            INNER JOIN claim_totalrestart ctr
                USING (claim_id)
             '''

In [28]:
# # execute sql
# cursor.execute(sql_file_tr)

In [29]:
# # save query results as DF
# df3 = pd.DataFrame(cursor.fetchall())

In [30]:
df3 = pd.DataFrame(mysql_q(vgc_u, vgc_pw, vgc_host, 'visualgap_claims', sql_file_tr, 0, 0))
df3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1490,202111092512,12,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,Check,Ralph,Sutton,1,4000.0,123-456,2021-10-15,jediracer13@gmail.com,
1,1471,202110182493,12,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,Check,ANDREW,FLYNN,1,0.0,0,2021-09-10,jediracer13@gmail.com,


In [31]:
# add column names
df3_cols = ['claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'dealer_securityId', 'contact', 'address1', 'city', 'state', 'zip', 
                            'pymt_method', 'first', 'last', 'pymt_type_id', 'amount', 'acct_number', 'loss_date', 'email', 'email2']

df3.columns = df3_cols

In [32]:
# # close mysql connection
# cursor.close()
# cnx.close()

In [33]:
# # get expense account name
# cnx = mc.connect(user=mysql_u, password=mysql_pw,
#                 host=mysql_host,
#                 database='claim_qb_payments')
# cursor = cnx.cursor()

In [34]:
# GAP
# convert columns list to string
cols = ", ".join(df_cols)

# insert DF into the ready_to_be_paid table of the claim_qb_payments database
for x,rows in df.iterrows():

    sql_file2 = '''INSERT INTO ready_to_be_paid ({columns}, payment_category_id, check_nbr, batch_id, qb_txnid, toVGC) VALUES ({claim_id}, "{claim_nbr}", {carrier_id},"{lender_name}", "{lender_id}","{contact}", 
                "{address1}", "{city}", "{state}", "{zip}", "{pymt_method}", "{first}", "{last}", {pymt_type_id}, {amount}, "{acct_nbr}", "{loss_date}", "{email}", "{email2}", 1, 0, {batchId}, 0, 0);'''.format(columns=cols, 
                claim_id=rows['claim_id'], claim_nbr=rows['claim_nbr'], carrier_id=rows['carrier_id'], lender_name=rows['lender_name'], lender_id=rows['dealer_securityId'], 
                contact=rows['contact'], address1=rows['address1'], city=rows['city'], state=rows['state'], zip=rows['zip'], 
                pymt_method=rows['pymt_method'], first = rows['first'], last = rows['last'], pymt_type_id = rows['pymt_type_id'], 
                amount = rows['amount'], acct_nbr = rows['acct_number'], loss_date = rows['loss_date'], email= rows['email'], email2= rows['email2'], batchId = batch_id)
     
    # # execute and commit sql
    # cursor.execute(sql_file2)
    # cnx.commit()
    mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', sql_file2, 0, 1)

In [35]:
# PLUS
# convert columns list to string
cols = ", ".join(df2_cols)

# insert DF into the ready_to_be_paid table of the claim_qb_payments database
for x,rows in df2.iterrows():
    sql_file2_plus = '''INSERT INTO ready_to_be_paid ({columns}, payment_category_id, check_nbr, batch_id, qb_txnid, toVGC) VALUES ({claim_id}, "{claim_nbr}", {carrier_id},"{lender_name}", "{lender_id}","{contact}", 
                "{address1}", "{city}", "{state}", "{zip}", "{pymt_method}", "{first}", "{last}", {pymt_type_id}, {amount}, "{acct_nbr}", "{loss_date}", "{email}", "{email2}", 2, 0, {batchId}, 0, 0);'''.format(columns=cols, 
                claim_id=rows['claim_id'], claim_nbr=rows['claim_nbr'], carrier_id=rows['carrier_id'], lender_name=rows['lender_name'], lender_id=rows['dealer_securityId'], 
                contact=rows['contact'], address1=rows['address1'], city=rows['city'], state=rows['state'], zip=rows['zip'], 
                pymt_method=rows['pymt_method'], first = rows['first'], last = rows['last'], pymt_type_id = rows['pymt_type_id'], 
                amount = rows['amount'], acct_nbr = rows['acct_number'], loss_date = rows['loss_date'], email= rows['email'], email2= rows['email2'], batchId = batch_id)
          
    # execute and commit sql
    # cursor.execute(sql_file2_plus)
    # cnx.commit()
    mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', sql_file2_plus, 0, 1)

In [36]:
# TOTALRESTART
# convert columns list to string
cols = ", ".join(df3_cols)

# insert DF into the ready_to_be_paid table of the claim_qb_payments database
for x,rows in df3.iterrows():
    sql_file2_tr = '''INSERT INTO ready_to_be_paid ({columns}, payment_category_id, check_nbr, batch_id, qb_txnid, toVGC) VALUES ({claim_id}, "{claim_nbr}", {carrier_id},"{lender_name}", "{lender_id}","{contact}", 
                "{address1}", "{city}", "{state}", "{zip}", "{pymt_method}", "{first}", "{last}", {pymt_type_id}, {amount}, "{acct_nbr}", "{loss_date}", "{email}", "{email2}", 3, 0, {batchId}, 0, 0);'''.format(columns=cols, 
                claim_id=rows['claim_id'], claim_nbr=rows['claim_nbr'], carrier_id=rows['carrier_id'], lender_name=rows['lender_name'], lender_id=rows['dealer_securityId'], 
                contact=rows['contact'], address1=rows['address1'], city=rows['city'], state=rows['state'], zip=rows['zip'], 
                pymt_method=rows['pymt_method'], first = rows['first'], last = rows['last'], pymt_type_id = rows['pymt_type_id'], 
                amount = rows['amount'], acct_nbr = rows['acct_number'], loss_date = rows['loss_date'], email= rows['email'], email2= rows['email2'], batchId = batch_id)
          
    # execute and commit sql
    # cursor.execute(sql_file2_tr)
    # cnx.commit()
    mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', sql_file2_tr, 0, 1)

In [37]:
# create select query to pull current batch with ID
sql_file3 = '''SELECT rtbp_id, claim_id, claim_nbr, carrier_id, lender_name, dealer_securityId, contact, address1, city, 
                     state, zip, pymt_method, first, last, pymt_type_id, amount, payment_category_id, check_nbr, batch_id, 
                     qb_txnid, acct_number, loss_date, email, email2
              FROM ready_to_be_paid
              WHERE batch_id = {batchId}
              ORDER BY payment_category_id, claim_id;'''.format(batchId = batch_id)

In [38]:
# execute sql
# cursor.execute(sql_file3)

In [39]:

# save query results as DF
pymts_df = pd.DataFrame(mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', sql_file3, 0, 0))
pymts_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,2301,1466,202109172488,9,Charlotte Metro Credit Union,22260,CMCU Contact,8829 Chapel Square Dr (Jareds Office),Cincinnati,OH,45249,Check,Sam,Hunt,1,5302.64,1,0,20220209162206,0,0,2021-09-02,,jared@visualgap.com
1,2304,1469,202110182491,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,TIFFANY,LEGO,1,2144.05,1,0,20220209162206,0,0,2021-08-28,jediracer13@gmail.com,
2,2307,1470,202110182492,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,MARY,SCHEIBLE,1,0.00,1,0,20220209162206,0,6343056-50,2021-10-06,jediracer13@gmail.com,
3,2302,1471,202110182493,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,ANDREW,FLYNN,1,0.00,1,0,20220209162206,0,0,2021-09-10,jediracer13@gmail.com,
4,2303,1472,202110182494,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,TYSHAUN,GRANT,1,5138.04,1,0,20220209162206,0,250510182,2021-08-30,jediracer13@gmail.com,
5,2305,1473,202110182495,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,ALEX,MITCHELL,1,0.00,1,0,20220209162206,0,0,2021-06-25,jediracer13@gmail.com,
6,2311,1474,202110192496,9,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,JOSHUA,WELCH,1,0.00,1,0,20220209162206,0,0,2021-09-10,jediracer13@gmail.com,
7,2306,1475,202110192497,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,OCTAVIO,PARTIDA,1,653.87,1,0,20220209162206,0,187895-01,2021-08-28,jediracer13@gmail.com,
8,2308,1476,202110192498,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,SHERRY,VOGT,1,0.00,1,0,20220209162206,0,136324-02L,2021-07-27,jediracer13@gmail.com,
9,2312,1477,202110192499,9,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,BRIAN,CHANDLER,2,191.17,1,0,20220209162206,0,250505961,2021-08-20,jediracer13@gmail.com,


In [40]:
# save query results as DF
# pymts_df = pd.DataFrame(cursor.fetchall())

# add column names
pymts_df_cols = ['rtbp_id', 'claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'dealer_securityId', 'contact', 'address1', 'city', 'state', 'zip', 
                    'pymt_method', 'first', 'last', 'pymt_type_id', 'amount','payment_category_id', 'check_nbr', 'batch_id', 'qb_txnid', 
                    'acct_number', 'loss_date', 'email', 'email2']

pymts_df.columns = pymts_df_cols

In [41]:
# sql query for expense accounts in DB
sql_file4 = '''
    SELECT carrier_id, qb_fullname
    FROM qb_accounts
    WHERE account_type = 'Expense';
    '''

In [42]:
# save query results as DF
expense_df = pd.DataFrame(mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', sql_file4, 0, 0))
expense_df

,0,1
0,9,GAP Claims Advance-ANICO
1,8,Claims Paid:Securian Claims
2,12,Claims Paid:TotalRestart


In [43]:
# # execute sql
# cursor.execute(sql_file4)
# # save query results as DF
# expense_df = pd.DataFrame(cursor.fetchall())
# add column names to DF
col_names = ['carrier_id', 'expense']
expense_df.columns = col_names

In [44]:
# sql query for checking accounts in DB
sql_file5 = '''
    SELECT carrier_id, qb_fullname
    FROM qb_accounts
    WHERE account_type = 'Checking';
    '''

In [45]:
# save query results as DF
checking_df = pd.DataFrame(mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', sql_file5, 0, 0))
checking_df

,0,1
0,9,Claims ANICO (Fifth Third)
1,8,Claims Securian
2,12,TotalRestart Claims


In [46]:
# # execute sql
# cursor.execute(sql_file5)
# # save query results as DF
# checking_df = pd.DataFrame(cursor.fetchall())
# add column names to DF
col_names = ['carrier_id', 'checking']
checking_df.columns = col_names

In [47]:
# Merge expense account name into df
pymts_df = pymts_df.merge(expense_df, how='left', left_on='carrier_id', right_on='carrier_id').copy()

In [48]:
# Merge checking account name into df
pymts_df = pymts_df.merge(checking_df, how='left', left_on='carrier_id', right_on='carrier_id').copy()

In [49]:
pymts_df

,rtbp_id,claim_id,claim_nbr,carrier_id,lender_name,dealer_securityId,contact,address1,city,state,zip,pymt_method,first,last,pymt_type_id,amount,payment_category_id,check_nbr,batch_id,qb_txnid,acct_number,loss_date,email,email2,expense,checking
0,2301,1466,202109172488,9,Charlotte Metro Credit Union,22260,CMCU Contact,8829 Chapel Square Dr (Jareds Office),Cincinnati,OH,45249,Check,Sam,Hunt,1,5302.64,1,0,20220209162206,0,0,2021-09-02,,jared@visualgap.com,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third)
1,2304,1469,202110182491,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,TIFFANY,LEGO,1,2144.05,1,0,20220209162206,0,0,2021-08-28,jediracer13@gmail.com,,Claims Paid:Securian Claims,Claims Securian
2,2307,1470,202110182492,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,MARY,SCHEIBLE,1,0.00,1,0,20220209162206,0,6343056-50,2021-10-06,jediracer13@gmail.com,,Claims Paid:Securian Claims,Claims Securian
3,2302,1471,202110182493,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,ANDREW,FLYNN,1,0.00,1,0,20220209162206,0,0,2021-09-10,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third)
4,2303,1472,202110182494,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,TYSHAUN,GRANT,1,5138.04,1,0,20220209162206,0,250510182,2021-08-30,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third)
5,2305,1473,202110182495,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,ALEX,MITCHELL,1,0.00,1,0,20220209162206,0,0,2021-06-25,jediracer13@gmail.com,,Claims Paid:Securian Claims,Claims Securian
6,2311,1474,202110192496,9,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,JOSHUA,WELCH,1,0.00,1,0,20220209162206,0,0,2021-09-10,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third)
7,2306,1475,202110192497,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,OCTAVIO,PARTIDA,1,653.87,1,0,20220209162206,0,187895-01,2021-08-28,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third)
8,2308,1476,202110192498,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,SHERRY,VOGT,1,0.00,1,0,20220209162206,0,136324-02L,2021-07-27,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third)
9,2312,1477,202110192499,9,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,BRIAN,CHANDLER,2,191.17,1,0,20220209162206,0,250505961,2021-08-20,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third)


In [50]:
# Create Error DF
error_df = pd.DataFrame(columns = ['rtbp_id', 'err_msg'])

In [51]:
# Check for missing expense and/or checking names
chk_exp_error_df = pymts_df.loc[(pymts_df['expense'].isnull()) | (pymts_df['checking'].isnull())]
chk_exp_error_df

,rtbp_id,claim_id,claim_nbr,carrier_id,lender_name,dealer_securityId,contact,address1,city,state,zip,pymt_method,first,last,pymt_type_id,amount,payment_category_id,check_nbr,batch_id,qb_txnid,acct_number,loss_date,email,email2,expense,checking
14,2315,1492,202112272514,6,Charlotte Metro Credit Union,22260,CMCU Contact,8829 Chapel Square Dr (Jareds Office),Cincinnati,OH,45249,Check,Supplemental,Letter,2,0.0,1,0,20220209162206,0,12456789,2021-12-01,,jared@visualgap.com,NaN,NaN


In [52]:
# Drop rows with error
pymts_df.drop(pymts_df[(pymts_df['expense'].isnull()) | (pymts_df['checking'].isnull())].index, inplace = True)
# pymts_df

In [53]:
# add to Error DF
if len(chk_exp_error_df) > 0:
    for index, row in chk_exp_error_df.iterrows():
        error_df.loc[error_df.shape[0]] = [row['rtbp_id'], 'No matching CHECKING and-or EXPENSE accounts in QuickBooks']
error_df        

,rtbp_id,err_msg
0,2315,No matching CHECKING and-or EXPENSE accounts i...


In [54]:
# Create sql server connection
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+svr+';DATABASE='+db+';UID='+sql_u+';PWD='+ sql_pw)

In [55]:
# create query
sql_svr_file = '''SELECT VGSecurityId, QB_ListID
                  FROM business_entity
                  WHERE QB_ListID IS NOT NULL;
               '''
# execute query
listid_df = pd.read_sql(sql_svr_file, cnxn) 

In [56]:
# TEMPORARY ########################################################################################
# Convert test dealer_securityId to Production dealer_securityId
pymts_df['dealer_securityId'].replace({22260:46724,21945:52715}, inplace=True)
# TEMPORARY ########################################################################################

In [57]:
# Merge QB_ListID into df
pymts_df = pymts_df.merge(listid_df, how='left', left_on='dealer_securityId', right_on='VGSecurityId').copy()

In [58]:
# Check for missing QBlistID
qbListId_error_df = pymts_df.loc[(pymts_df['QB_ListID'].isnull())]
qbListId_error_df

,rtbp_id,claim_id,claim_nbr,carrier_id,lender_name,dealer_securityId,contact,address1,city,state,zip,pymt_method,first,last,pymt_type_id,amount,payment_category_id,check_nbr,batch_id,qb_txnid,acct_number,loss_date,email,email2,expense,checking,VGSecurityId,QB_ListID


In [59]:
# add to Error DF
if len(qbListId_error_df) > 0:
    for index, row in qbListId_error_df.iterrows():
        error_df.loc[error_df.shape[0]] = [row['rtbp_id'], 'Missing QB_LIST_ID in the Claim_Payments database.  Run GET CUSTOMER to update the database.']
error_df  

,rtbp_id,err_msg
0,2315,No matching CHECKING and-or EXPENSE accounts i...


In [60]:
# Using error_df, send error messages to ready_to_be_paid DB and update toVGC = 2

# connect to claim_qb_payments db
# cnx = mc.connect(user=mysql_u, password=mysql_pw,
#                 host=mysql_host,
#                 database='claim_qb_payments')
# cursor = cnx.cursor()

if len(error_df) > 0:
    for index, row in error_df.iterrows():
        err_sql = '''
                  UPDATE ready_to_be_paid
                  SET toVGC = 2,
                      err_msg = '{err_msg}'
                  WHERE rtbp_id = {rtbp_id};
                  '''.format(err_msg=row['err_msg'], rtbp_id=row['rtbp_id'])
#         # Run SQL
#         cursor.execute(err_sql)
#         cnx.commit()

# # close mysql connection
# cursor.close()
# cnx.close()

mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', err_sql, 0, 1)

0

In [61]:
# Drop rows with error
pymts_df.drop(pymts_df[pymts_df['QB_ListID'].isnull()].index, inplace = True)
pymts_df

,rtbp_id,claim_id,claim_nbr,carrier_id,lender_name,dealer_securityId,contact,address1,city,state,zip,pymt_method,first,last,pymt_type_id,amount,payment_category_id,check_nbr,batch_id,qb_txnid,acct_number,loss_date,email,email2,expense,checking,VGSecurityId,QB_ListID
0,2301,1466,202109172488,9,Charlotte Metro Credit Union,46724,CMCU Contact,8829 Chapel Square Dr (Jareds Office),Cincinnati,OH,45249,Check,Sam,Hunt,1,5302.64,1,0,20220209162206,0,0,2021-09-02,,jared@visualgap.com,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724
1,2304,1469,202110182491,8,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,TIFFANY,LEGO,1,2144.05,1,0,20220209162206,0,0,2021-08-28,jediracer13@gmail.com,,Claims Paid:Securian Claims,Claims Securian,52715,80000002-1628088752
2,2307,1470,202110182492,8,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,MARY,SCHEIBLE,1,0.00,1,0,20220209162206,0,6343056-50,2021-10-06,jediracer13@gmail.com,,Claims Paid:Securian Claims,Claims Securian,52715,80000002-1628088752
3,2302,1471,202110182493,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,ANDREW,FLYNN,1,0.00,1,0,20220209162206,0,0,2021-09-10,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724
4,2303,1472,202110182494,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,TYSHAUN,GRANT,1,5138.04,1,0,20220209162206,0,250510182,2021-08-30,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724
5,2305,1473,202110182495,8,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,ALEX,MITCHELL,1,0.00,1,0,20220209162206,0,0,2021-06-25,jediracer13@gmail.com,,Claims Paid:Securian Claims,Claims Securian,52715,80000002-1628088752
6,2311,1474,202110192496,9,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,JOSHUA,WELCH,1,0.00,1,0,20220209162206,0,0,2021-09-10,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),52715,80000002-1628088752
7,2306,1475,202110192497,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,OCTAVIO,PARTIDA,1,653.87,1,0,20220209162206,0,187895-01,2021-08-28,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724
8,2308,1476,202110192498,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,SHERRY,VOGT,1,0.00,1,0,20220209162206,0,136324-02L,2021-07-27,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724
9,2312,1477,202110192499,9,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,BRIAN,CHANDLER,2,191.17,1,0,20220209162206,0,250505961,2021-08-20,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),52715,80000002-1628088752


In [62]:
# Add carrier name
# cnx_c = mc.connect(user=vgc_u, password=vgc_pw,
#                 host=vgc_host,
#                 database='visualgap_claims')
# cursor_c = cnx_c.cursor()

sql_file6 = '''
    SELECT carrier_id, description
    FROM carriers;
    '''
# cursor_c.execute(sql_file6)

# carrier_df = pd.DataFrame(cursor_c.fetchall())

# save query results as DF
carrier_df = pd.DataFrame(mysql_q(vgc_u, vgc_pw, vgc_host, 'visualgap_claims', sql_file6, 0, 0))

col_names = ['carrier_id', 'carrier']
carrier_df.columns = col_names

# cursor_c.close()
# cnx_c.close()

# Merge QB_ListID into df
pymts_df = pymts_df.merge(carrier_df, left_on='carrier_id', right_on='carrier_id').copy()

In [63]:
# payments greater than 0 df
qb_pymts_df = pymts_df.loc[pymts_df['amount'] > 0].copy()


In [64]:
qb_pymts_df

,rtbp_id,claim_id,claim_nbr,carrier_id,lender_name,dealer_securityId,contact,address1,city,state,zip,pymt_method,first,last,pymt_type_id,amount,payment_category_id,check_nbr,batch_id,qb_txnid,acct_number,loss_date,email,email2,expense,checking,VGSecurityId,QB_ListID,carrier
0,2301,1466,202109172488,9,Charlotte Metro Credit Union,46724,CMCU Contact,8829 Chapel Square Dr (Jareds Office),Cincinnati,OH,45249,Check,Sam,Hunt,1,5302.64,1,0,20220209162206,0,0,2021-09-02,,jared@visualgap.com,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
2,2303,1472,202110182494,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,TYSHAUN,GRANT,1,5138.04,1,0,20220209162206,0,250510182,2021-08-30,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
4,2306,1475,202110192497,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,OCTAVIO,PARTIDA,1,653.87,1,0,20220209162206,0,187895-01,2021-08-28,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
6,2312,1477,202110192499,9,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,BRIAN,CHANDLER,2,191.17,1,0,20220209162206,0,250505961,2021-08-20,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),52715,80000002-1628088752,ANICO
7,2313,1488,202110272510,9,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,TYSHAUN,GRANT,1,3498.00,1,0,20220209162206,0,250510182,2021-08-30,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),52715,80000002-1628088752,ANICO
9,2317,1475,202110192497,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,OCTAVIO,PARTIDA,1,1000.00,2,0,20220209162206,0,187895-01,2021-08-28,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
10,2304,1469,202110182491,8,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,TIFFANY,LEGO,1,2144.05,1,0,20220209162206,0,0,2021-08-28,jediracer13@gmail.com,,Claims Paid:Securian Claims,Claims Securian,52715,80000002-1628088752,Securian Casualty
13,2309,1478,202110192500,8,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,ASHLEY,MAPP,1,2820.16,1,0,20220209162206,0,22004112-030,2021-09-18,jediracer13@gmail.com,,Claims Paid:Securian Claims,Claims Securian,52715,80000002-1628088752,Securian Casualty
14,2310,1479,202110192501,8,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,LEELAND,WATKINS,1,103.35,1,0,20220209162206,0,804240-12,2021-08-18,jediracer13@gmail.com,,Claims Paid:Securian Claims,Claims Securian,52715,80000002-1628088752,Securian Casualty
15,2316,1473,202110182495,8,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,ALEX,MITCHELL,1,1000.00,2,0,20220209162206,0,0,2021-06-25,jediracer13@gmail.com,,Claims Paid:Securian Claims,Claims Securian,52715,80000002-1628088752,Securian Casualty


In [65]:
# Connect to Quickbooks
try:
    sessionManager = wc.Dispatch("QBXMLRP2.RequestProcessor")    
    sessionManager.OpenConnection('', 'Claim Payments')
    ticket = sessionManager.BeginSession("", 2)
except Exception as e:
    print('''
    Make sure QuickBooks is running and you are logged into the Company File.
    ERROR: {}'''.format(e))
    sys.exit("Error with communicating with QuickBooks")


In [66]:
# create qbxml query to add payments to QB
pay_date = f"{dt.date.today():%Y-%m-%d}"

for index, row in qb_pymts_df.iterrows():
    if row['payment_category_id'] == 1:
        if row['pymt_type_id'] == 2:
            pymt_type = 'Additional GAP Claim Pymt'
        else: 
            pymt_type = 'GAP Claim'

    elif row['payment_category_id'] == 2:
        if row['pymt_type_id'] == 2:
            pymt_type = 'Additional GAP Plus Pymt'
        else: 
            pymt_type = 'GAP Plus'
            
    elif row['payment_category_id'] == 3:
        if row['pymt_type_id'] == 2:
            pymt_type = 'Additional TotalRestart Pymt'
        else: 
            pymt_type = 'TotalRestart'

    pymtAmt = "{:.2f}".format(row['amount'])

    if row['pymt_method'] == 'Check':
        qbxmlQuery = '''
        <?qbxml version="14.0"?>
        <QBXML>
            <QBXMLMsgsRq onError="stopOnError">
                <CheckAddRq>
                    <CheckAdd>
                        <AccountRef>
                            <FullName>{checking}</FullName>
                        </AccountRef>
                        <PayeeEntityRef>
                            <ListID>{lender_qbid}</ListID>
                        </PayeeEntityRef>
                        <TxnDate>{date}</TxnDate>
                        <Memo>{memo}</Memo>
                        <Address>
                            <Addr1>{lender}</Addr1>
                            <Addr2>{contact}</Addr2>
                            <Addr3>{address}</Addr3>
                            <City>{city}</City>
                            <State>{state}</State>
                            <PostalCode>{zip}</PostalCode>
                        </Address>
                        <IsToBePrinted>true</IsToBePrinted>
                        <ExpenseLineAdd>
                            <AccountRef>
                                <FullName>{expense}</FullName>
                            </AccountRef>
                            <Amount>{amount}</Amount>
                            <Memo>{memo}</Memo>
                        </ExpenseLineAdd>
                    </CheckAdd>
                </CheckAddRq>
            </QBXMLMsgsRq>
        </QBXML>'''.format(checking=row['checking'], lender_qbid=row['QB_ListID'], lender=row['lender_name'], date=pay_date, 
                memo=f"{row['last']}/{row['first']} {pymt_type}", contact=row['contact'], address=row['address1'], city=row['city'], state=row['state'],
                zip=row['zip'], expense=row['expense'], amount = pymtAmt)

    elif 'ACH' in row['pymt_method']:
        qbxmlQuery = '''
        <?qbxml version="14.0"?>
        <QBXML>
            <QBXMLMsgsRq onError="stopOnError">
                <CheckAddRq>
                    <CheckAdd>
                        <AccountRef>
                            <FullName>{checking}</FullName>
                        </AccountRef>
                        <PayeeEntityRef>
                            <ListID>{lender_qbid}</ListID>
                        </PayeeEntityRef>
                        <RefNumber>ACH</RefNumber>
                        <TxnDate>{date}</TxnDate>
                        <Memo>{memo}</Memo>
                        <Address>
                            <Addr1>{lender}</Addr1>
                            <Addr2>{contact}</Addr2>
                            <Addr3>{address}</Addr3>
                            <City>{city}</City>
                            <State>{state}</State>
                            <PostalCode>{zip}</PostalCode>
                        </Address>
                        <IsToBePrinted>false</IsToBePrinted>
                        <ExpenseLineAdd>
                            <AccountRef>
                                <FullName>{expense}</FullName>
                            </AccountRef>
                            <Amount>{amount}</Amount>
                            <Memo>{memo}</Memo>
                        </ExpenseLineAdd>
                    </CheckAdd>
                </CheckAddRq>            
            </QBXMLMsgsRq>
        </QBXML>'''.format(checking=row['checking'], lender_qbid=row['QB_ListID'], lender=row['lender_name'], date=pay_date, 
                memo=f"{row['last']}/{row['first']} {pymt_type}", contact=row['contact'], address=row['address1'], city=row['city'], state=row['state'],
                zip=row['zip'], expense=row['expense'], amount = pymtAmt)
        
    # Send query and receive response
    responseString = sessionManager.ProcessRequest(ticket, qbxmlQuery)

    # output TxnID
    QBXML = ET.fromstring(responseString)
    QBXMLMsgsRs = QBXML.find('QBXMLMsgsRs')
    checkResults = QBXMLMsgsRs.iter("CheckRet")
    txnId = 0
    for checkResult in checkResults:
        txnId = checkResult.find('TxnID').text

    # Add TxnID to ready_to_be_paid table
    qb_sql_file = '''UPDATE ready_to_be_paid
                   SET qb_txnid = '{TxnID}'
                   WHERE rtbp_id = {rowID};'''.format(TxnID=txnId, rowID=row['rtbp_id'])
    
    # execute and commit sql
    mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', qb_sql_file, 0, 1)
    # cursor.execute(sql_file6)
    # cnx.commit()


In [67]:
# Disconnect from Quickbooks
sessionManager.EndSession(ticket)
sessionManager.CloseConnection()

In [68]:
# close mysql connection
# cursor.close()
# cnx.close()

In [69]:
# Add Fraud Language fields to pymts_df
# cnx_f = mc.connect(user=vgc_u, password=vgc_pw,
#                 host=vgc_host,
#                 database='visualgap_claims')
# cursor_f = cnx_f.cursor()

sql_file7 = '''
    SELECT StateId, 
    StateDesc,
    StateCode,
    CAST(Language AS CHAR(1000) CHARACTER SET utf8)
    FROM FraudLang;
    '''
# cursor_f.execute(sql_file7)

# fraud_lang_df = pd.DataFrame(cursor_f.fetchall())

# run sql query
fraud_lang_df = pd.DataFrame(mysql_q(vgc_u, vgc_pw, vgc_host, 'visualgap_claims', sql_file7, 0, 0))

col_names = ['StateId', 'StateDesc', 'StateCode', 'f_lang']
fraud_lang_df.columns = col_names

# cursor_f.close()
# cnx_f.close()

# Merge QB_ListID into df
pymts_df = pymts_df.merge(fraud_lang_df, how='left', left_on='state', right_on='StateId').copy()


In [70]:
# replace NaN with ''
pymts_df.fillna('', inplace = True)

In [71]:
# Update Column names
pymts_df.rename(columns = {'amount':'payment_amount', 'lender_name':'alt_name'}, inplace = True)

In [72]:
pymts_df

,rtbp_id,claim_id,claim_nbr,carrier_id,alt_name,dealer_securityId,contact,address1,city,state,zip,pymt_method,first,last,pymt_type_id,payment_amount,payment_category_id,check_nbr,batch_id,qb_txnid,acct_number,loss_date,email,email2,expense,checking,VGSecurityId,QB_ListID,carrier,StateId,StateDesc,StateCode,f_lang
0,2301,1466,202109172488,9,Charlotte Metro Credit Union,46724,CMCU Contact,8829 Chapel Square Dr (Jareds Office),Cincinnati,OH,45249,Check,Sam,Hunt,1,5302.64,1,0,20220209162206,0,0,2021-09-02,,jared@visualgap.com,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO,OH,Ohio,3999.21,"Any person who, with intent to defraud or know..."
1,2302,1471,202110182493,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,ANDREW,FLYNN,1,0.00,1,0,20220209162206,0,0,2021-09-10,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO,,,,
2,2303,1472,202110182494,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,TYSHAUN,GRANT,1,5138.04,1,0,20220209162206,0,250510182,2021-08-30,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO,,,,
3,2311,1474,202110192496,9,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,JOSHUA,WELCH,1,0.00,1,0,20220209162206,0,0,2021-09-10,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),52715,80000002-1628088752,ANICO,TX,Texas,V.T.C.A. Ins. Code s. 704.00,Any person who knowingly presents a false or f...
4,2306,1475,202110192497,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,OCTAVIO,PARTIDA,1,653.87,1,0,20220209162206,0,187895-01,2021-08-28,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO,,,,
5,2308,1476,202110192498,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,SHERRY,VOGT,1,0.00,1,0,20220209162206,0,136324-02L,2021-07-27,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO,,,,
6,2312,1477,202110192499,9,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,BRIAN,CHANDLER,2,191.17,1,0,20220209162206,0,250505961,2021-08-20,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),52715,80000002-1628088752,ANICO,TX,Texas,V.T.C.A. Ins. Code s. 704.00,Any person who knowingly presents a false or f...
7,2313,1488,202110272510,9,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,TYSHAUN,GRANT,1,3498.00,1,0,20220209162206,0,250510182,2021-08-30,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),52715,80000002-1628088752,ANICO,TX,Texas,V.T.C.A. Ins. Code s. 704.00,Any person who knowingly presents a false or f...
8,2314,1490,202111092512,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,Ralph,Sutton,1,0.00,1,0,20220209162206,0,123-456,2021-10-15,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO,,,,
9,2317,1475,202110192497,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,OCTAVIO,PARTIDA,1,1000.00,2,0,20220209162206,0,187895-01,2021-08-28,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO,,,,


Create GAP Letters

In [73]:
# Collect GAP payments
gap_pymts_df = pymts_df.loc[pymts_df['payment_category_id'] == 1].copy()
gap_letters_df = gap_pymts_df.loc[gap_pymts_df['payment_amount'] > 0].copy()

In [74]:
# Remove files from staging directory
file_staging_dir = './letters/staging/'
file_ext = ".pdf"

clear_dir(file_staging_dir, file_ext)

create letters for amounts greater than $0 paid via check

In [76]:
# Filter gap_letters_df for pymt_method 'Check'
checks_df = gap_letters_df.loc[gap_letters_df['pymt_method'] == 'Check'].copy()

if len(checks_df.index) > 0:
    # Create list of Carriers on checks_df
    carriers = []
    carriers = checks_df.carrier.unique()
    letter_cols = ['claim_nbr', 'loss_date', 'alt_name', 'contact', 'address1', 'city', 'state', 'zip', 'first', 'last', 'acct_number', 'payment_amount', 'StateDesc', 'StateCode', 'f_lang' ]

    # Loop through carriers list
    for carrier in carriers:

        # Variable Defaults
        sql_where_cal = ''
        g_letters = True
        s_letters = True
        cals = []

        # check for payment_type_id for GAP letters by carrier
        g_letters_df = checks_df.loc[(checks_df['pymt_type_id'] == 1) & (checks_df['carrier'] == carrier)].copy()

        # GAP Letter - create WHERE statement
        if len(g_letters_df.index) > 0:
            g_letters = True
        else:
            # No Letters
            g_letters = False

        # check for payment_type_id for Supplemental letters
        s_letters_df = checks_df.loc[(checks_df['pymt_type_id'] == 2) & (checks_df['carrier'] == carrier)].copy()

        # Supplemental Letter
        if len(s_letters_df.index) > 0:
            s_letters = True
        else:
            # No Letters
            s_letters = False

        # Calculations
        # check for calculations for carrier
        calculations_df = checks_df.loc[checks_df['carrier'] == carrier].copy()

        if len(calculations_df.index) > 0:
            # Multiple Calculation
            for index, row in calculations_df.iterrows():
                cals.append(row['claim_id'])

        # Connect to SQL DB
        # cnx = mc.connect(user=vgc_u, password=vgc_pw,
        #                 host=vgc_host,
        #                 database='visualgap_claims')
        # cursor = cnx.cursor()

        # GAP - Create letters
        if g_letters == True:
            # create df for template
            g_template_df = g_letters_df[letter_cols]

            # Create GAP Letters
            g_pdf_template = "letters/pdf_templates/GAP_letter_template.pdf"
            position = 1
            gap_letter(g_template_df, g_pdf_template, position)
        
        # Supplemental - Create letters
        if s_letters == True:
            # create df for template
            s_template_df = s_letters_df[letter_cols]
            
            # Create Supplement Letters
            s_pdf_template = "letters/pdf_templates/Supp_letter_template.pdf"
            position = 2
            gap_letter(s_template_df, s_pdf_template, position)

        # Calculations - Create SQL query, run query, create calculation sheets
        if g_letters == True or s_letters == True:

            # create calculation dataframe
            temp_cols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]                             
            c_consolidate_df = pd.DataFrame(columns = temp_cols)

            for cal in cals:
                # create sql for calculation
                c_sql_query = '''
                        SELECT c.claim_nbr, c.loss_date, l.alt_name, l.contact, b.first, b.last, cc.gap_payable,
                            cl.incp_date, cl.last_payment, cl.interest_rate, cl.amount AS Amt_Fin, cc.balance_last_pay,
                            cc.nbr_of_days, cc.per_day, ROUND(cc.payoff,2) AS payoff, (cc.ltv * 100) AS ltv, cc.covered_fin_amount,
                            (cc.percent_uncovered * 100) AS percent_uncovered, (cc.ltv_limit * 100) AS ltv_limit, v.nada_value, CONCAT(v.year, ' ', v.make, ' ', v.model) AS vehicle,
                            v.deductible, cls.description AS loss_type, cc26.description AS primary_carrier,
                            cc14.amount AS past_due, cc15.amount AS late_fees, cc16.amount AS skip_pymts,
                            cc17.amount AS skip_fees, cc5.amount AS primary_pymt, cc7.amount AS excess_deductible,
                            cc8.amount AS scr, cc9.amount AS clr, cc10.amount AS cdr, cc11.amount AS oref,
                            cc18.amount AS salvage, cc19.amount AS prior_dmg, cc20.amount AS over_ltv,
                            cc21.amount AS other1_amt, cc21.description AS other1_description,
                            cc22.amount AS other2_amt, cc22.description AS other2_description, ca.description AS carrier,
                            (cc22.amount + cc21.amount + cc20.amount + cc19.amount + cc18.amount + cc11.amount + cc10.amount + cc9.amount + 
                            cc8.amount + cc7.amount + cc5.amount + cc17.amount + cc16.amount + cc15.amount + cc14.amount) as subtotal
                        FROM claims c
                        INNER JOIN claim_lender l
                            USING (claim_id)
                        INNER JOIN claim_borrower b
                            USING (claim_id)
                        INNER JOIN claim_loan cl
                            USING (claim_id)
                        INNER JOIN claim_calculations cc
                            USING (claim_id)
                        INNER JOIN claim_vehicle v
                            USING (claim_id)
                        INNER JOIN claims_loss_type cls
                            USING (loss_type_id)
                        INNER JOIN carriers ca
                            ON c.carrier_id = ca.carrier_id                        
                        INNER JOIN claim_checklist AS cc26
                            ON c.claim_id = cc26.claim_id
                            AND cc26.checklist_item_id = 26    
                        INNER JOIN claim_checklist AS cc14
                            ON c.claim_id = cc14.claim_id
                            AND cc14.checklist_item_id = 14    
                        INNER JOIN claim_checklist AS cc15
                            ON c.claim_id = cc15.claim_id
                            AND cc15.checklist_item_id = 15    
                        INNER JOIN claim_checklist AS cc16
                            ON c.claim_id = cc16.claim_id
                            AND cc16.checklist_item_id = 16       
                        INNER JOIN claim_checklist AS cc17
                            ON c.claim_id = cc17.claim_id
                            AND cc17.checklist_item_id = 17       
                        INNER JOIN claim_checklist AS cc5
                            ON c.claim_id = cc5.claim_id
                            AND cc5.checklist_item_id = 5
                        INNER JOIN claim_checklist AS cc7
                            ON c.claim_id = cc7.claim_id
                            AND cc7.checklist_item_id = 7
                        INNER JOIN claim_checklist AS cc8
                            ON c.claim_id = cc8.claim_id
                            AND cc8.checklist_item_id = 8
                        INNER JOIN claim_checklist AS cc9
                            ON c.claim_id = cc9.claim_id
                            AND cc9.checklist_item_id = 9
                        INNER JOIN claim_checklist AS cc10
                            ON c.claim_id = cc10.claim_id
                            AND cc10.checklist_item_id = 10
                        INNER JOIN claim_checklist AS cc11
                            ON c.claim_id = cc11.claim_id
                            AND cc11.checklist_item_id = 11      
                        INNER JOIN claim_checklist AS cc18
                            ON c.claim_id = cc18.claim_id
                            AND cc18.checklist_item_id = 18
                        INNER JOIN claim_checklist AS cc19
                            ON c.claim_id = cc19.claim_id
                            AND cc19.checklist_item_id = 19
                        INNER JOIN claim_checklist AS cc20
                            ON c.claim_id = cc20.claim_id
                            AND cc20.checklist_item_id = 20     
                        INNER JOIN claim_checklist AS cc21
                            ON c.claim_id = cc21.claim_id
                            AND cc21.checklist_item_id = 21
                        INNER JOIN claim_checklist AS cc22
                            ON c.claim_id = cc22.claim_id
                            AND cc22.checklist_item_id = 22  
                        WHERE c.claim_id = {claimID};'''.format(claimID = cal)

                # cursor.execute(c_sql_query)
                # save query results as DF
                # c_temp_df = pd.DataFrame(cursor.fetchall())
                c_temp_df = pd.DataFrame(mysql_q(vgc_u, vgc_pw, vgc_host, 'visualgap_claims', c_sql_query, 0, 0))

                # append query result to c_template_df
                c_consolidate_df = c_consolidate_df.append(c_temp_df)

            cal_cols = {0:'claim_nbr', 1:'loss_date', 2:'alt_name', 3:'contact', 4:'first', 5:'last', 6:'gap_payable', 7:'incp_date', 8:'last_payment', 9:'interest_rate', 10:'Amt_Fin',
                11:'balance_last_pay', 12:'nbr_of_days', 13:'per_day', 14:'payoff', 15:'ltv', 16:'covered_fin_amount', 17:'percent_uncovered', 18:'ltv_limit', 19:'nada_value',
                20:'vehicle', 21:'deductible', 22:'loss_type', 23:'primary_carrier', 24:'past_due', 25:'late_fees', 26:'skip_pymts', 27:'skip_fees', 28:'primary_pymt',
                29:'excess_deductible', 30:'scr', 31:'clr', 32:'cdr', 33:'oref', 34:'salvage', 35:'prior_dmg', 36:'over_ltv', 37:'other1_amt', 38:'other1_description',
                39:'other2_amt', 40:'other2_description', 41:'carrier', 42:'subtotal'}

            c_template_df = c_consolidate_df.rename(columns = cal_cols)
            c_template_df = c_template_df.reset_index(drop=True)

            # Create Calculations
            c_pdf_template = "letters/pdf_templates/GAP_calculation_template.pdf"
            position = 3
            calculations(c_template_df, c_pdf_template, position)
                
        # Close SQL Connection
        # cursor.close()
        # cnx.close()

        # create a list of file to concatenate
        file_list = fileList(file_staging_dir, file_ext)

        # Concatenated output file
        outfn = f'S:/claims/letters/{carrier}_{now.strftime("%Y-%m-%d")}_GAP'

        ConCat_pdf(file_list, outfn)

        # Remove files from staging directory
        clear_dir(file_staging_dir, file_ext)

else: print('No amount greater then 0.')


C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_df['payment_amount'] = template_df['payment_amount'].map('${:,.2f}'.format)
C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_df['loss_date'] = pd.to_datetime(template_df['loss_date']).dt.strftime('%B %d, %Y')
C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:7: SettingWithCopyWarning: 
A value is

In [77]:
# Update toVGC to 1
update_tovgc_1(checks_df)

Create Plus Letters

In [78]:
# Collect GAP Plus
plus_pymts_df = pymts_df.loc[pymts_df['payment_category_id'] == 2].copy()
# plus_pymts_df.rename(columns = {'amount':'payment_amount', 'lender_name':'alt_name'}, inplace = True)
plus_letters_df = plus_pymts_df.loc[plus_pymts_df['payment_amount'] > 0].copy()

In [79]:
# Filter plus_letters_df for pymt_method 'Check'
plus_df = plus_letters_df.loc[plus_letters_df['pymt_method'] == 'Check'].copy()

if len(plus_df.index) > 0:

    # Create list of Carriers on checks_df
    p_carriers = []
    p_carriers = plus_df.carrier.unique()

    # Loop through carriers list
    for p_carrier in p_carriers:

        # Variable Defaults
        p_letters = True

        # check for payment_type_id for GAP letters by carrier
        p_letters_df = plus_df.loc[plus_df['carrier'] == p_carrier].copy()

        # check for records
        if len(p_letters_df.index) > 0:
            p_letters = True
        else:
            # No Letters
            p_letters = False

        # PLUS - Create letters
        if p_letters == True:
            # create df for template
            p_template_df = p_letters_df[letter_cols]

            # Create GAP Letters
            p_pdf_template = "letters/pdf_templates/PLUS_letter_template.pdf"
            position = 1
            gap_letter(p_template_df, p_pdf_template, position)
        
            file_list = fileList(file_staging_dir, file_ext)

            # Concatenated output file
            outfn = f'S:/claims/letters/{p_carrier}_{now.strftime("%Y-%m-%d")}_PLUS'

            ConCat_pdf(file_list, outfn)

        # Remove files from staging directory
        clear_dir(file_staging_dir, file_ext)

else: print('No Plus.')

C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_df['payment_amount'] = template_df['payment_amount'].map('${:,.2f}'.format)
C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_df['loss_date'] = pd.to_datetime(template_df['loss_date']).dt.strftime('%B %d, %Y')
C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:7: SettingWithCopyWarning: 
A value is

In [80]:
# Update toVGC to 1
update_tovgc_1(plus_df)

Create TotalRestart Letters

In [81]:
# Collect GAP Plus
tr_pymts_df = pymts_df.loc[pymts_df['payment_category_id'] == 3].copy()
# plus_pymts_df.rename(columns = {'amount':'payment_amount', 'lender_name':'alt_name'}, inplace = True)
tr_letters_df = tr_pymts_df.loc[tr_pymts_df['payment_amount'] > 0].copy()

In [83]:
# Filter tr_letters_df for pymt_method 'Check'
tr_df = tr_letters_df.loc[tr_letters_df['pymt_method'] == 'Check'].copy()

if len(tr_df.index) > 0:
    # Create list of Carriers on checks_df
    tr_carriers = []
    tr_carriers = tr_df.carrier.unique()

    # Loop through carriers list
    for tr_carrier in tr_carriers:

        # Variable Defaults
        tr_sql_where_cal = ''
        tr_letters = True
        tr_s_letters = True
        tr_cals = []

        # check for payment_type_id for TR letters by carrier
        tr_letters_df = tr_df.loc[(tr_df['pymt_type_id'] == 1) & (tr_df['carrier'] == tr_carrier)].copy()

        # TR Letter - create WHERE statement
        if len(tr_letters_df.index) > 0:
            tr_letters = True
        else:
            # No Letters
            tr_letters = False

        # check for payment_type_id for Supplemental letters
        tr_s_letters_df = tr_df.loc[(tr_df['pymt_type_id'] == 2) & (tr_df['carrier'] == tr_carrier)].copy()

        # Supplemental Letter
        if len(tr_s_letters_df.index) > 0:
            tr_s_letters = True
        else:
            # No Letters
            tr_s_letters = False

        # Calculations
        # check for calculations for carrier
        tr_calculations_df = tr_df.loc[tr_df['carrier'] == tr_carrier].copy()

        if len(tr_calculations_df.index) > 0:
            # Multiple Calculation
            for index, row in tr_calculations_df.iterrows():
                tr_cals.append(row['claim_id'])

        # Connect to SQL DB
        # cnx = mc.connect(user=vgc_u, password=vgc_pw,
        #                 host=vgc_host,
        #                 database='visualgap_claims')
        # cursor = cnx.cursor()

        # TR - Create letters
        if tr_letters == True:
            # create df for template
            tr_template_df = tr_letters_df[letter_cols]

            # Create TR Letters
            tr_pdf_template = "letters/pdf_templates/TR_letter_template.pdf"
            position = 1
            gap_letter(tr_template_df, tr_pdf_template, position)
        
        # Supplemental - Create letters
        if tr_s_letters == True:
            # create df for template
            tr_s_template_df = tr_s_letters_df[letter_cols]
            
            # Create Supplement Letters
            tr_s_pdf_template = "letters/pdf_templates/TR_letter_template.pdf"
            position = 2
            gap_letter(tr_s_template_df, tr_s_pdf_template, position)

        # Calculations - Create SQL query, run query, create calculation sheets
        if tr_letters == True or tr_s_letters == True:

            # create calculation dataframe
            temp_cols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]                             
            tr_consolidate_df = pd.DataFrame(columns = temp_cols)

            for tr_cal in tr_cals:
                # create sql for calculation
                tr_sql_query = '''
                        SELECT c.claim_nbr, c.loss_date, l.alt_name, l.contact, b.first, b.last, cc.gap_payable,
                            cl.incp_date, v.nada_value, CONCAT(v.year, ' ', v.make, ' ', v.model) AS vehicle,
                            cls.description AS loss_type, cc26.description AS primary_carrier, tr.max_benefit,
                            cc5.amount AS primary_pymt, cc7.amount AS excess_deductible, tr.term, tr.totalrestart_payable,
                            cc8.amount AS scr, cc9.amount AS clr, cc10.amount AS cdr, cc11.amount AS oref,
                            cc18.amount AS salvage, cc19.amount AS prior_dmg, 
                            cc21.amount AS other1_amt, cc21.description AS other1_description,
                            cc22.amount AS other2_amt, cc22.description AS other2_description, 
                            cc35.amount AS other3_amt, cc35.description AS other3_description, ca.description AS carrier,
                            (cc5.amount + cc7.amount + cc18.amount + cc19.amount + cc.gap_payable + cc8.amount + cc8.amount + 
                            cc9.amount + cc10.amount + cc11.amount + cc21.amount + cc22.amount + cc35.amount) as subtotal
                        FROM claims c
                        INNER JOIN claim_lender l
                            USING (claim_id)
                        INNER JOIN claim_borrower b
                            USING (claim_id)
                        INNER JOIN claim_loan cl
                            USING (claim_id)
                        INNER JOIN claim_calculations cc
                            USING (claim_id)
                        INNER JOIN claim_vehicle v
                            USING (claim_id)
                        INNER JOIN claims_loss_type cls
                            USING (loss_type_id)
                        INNER JOIN carriers ca
                            ON c.carrier_id = ca.carrier_id                        
                        INNER JOIN claim_checklist AS cc26
                            ON c.claim_id = cc26.claim_id
                            AND cc26.checklist_item_id = 26                  
                        INNER JOIN claim_checklist AS cc5
                            ON c.claim_id = cc5.claim_id
                            AND cc5.checklist_item_id = 5
                        INNER JOIN claim_checklist AS cc7
                            ON c.claim_id = cc7.claim_id
                            AND cc7.checklist_item_id = 7
                        INNER JOIN claim_checklist AS cc8
                            ON c.claim_id = cc8.claim_id
                            AND cc8.checklist_item_id = 8
                        INNER JOIN claim_checklist AS cc9
                            ON c.claim_id = cc9.claim_id
                            AND cc9.checklist_item_id = 9
                        INNER JOIN claim_checklist AS cc10
                            ON c.claim_id = cc10.claim_id
                            AND cc10.checklist_item_id = 10
                        INNER JOIN claim_checklist AS cc11
                            ON c.claim_id = cc11.claim_id
                            AND cc11.checklist_item_id = 11      
                        INNER JOIN claim_checklist AS cc18
                            ON c.claim_id = cc18.claim_id
                            AND cc18.checklist_item_id = 18
                        INNER JOIN claim_checklist AS cc19
                            ON c.claim_id = cc19.claim_id
                            AND cc19.checklist_item_id = 19     
                        INNER JOIN claim_checklist AS cc21
                            ON c.claim_id = cc21.claim_id
                            AND cc21.checklist_item_id = 21
                        INNER JOIN claim_checklist AS cc22
                            ON c.claim_id = cc22.claim_id
                            AND cc22.checklist_item_id = 22
                        INNER JOIN claim_checklist AS cc35
                            ON c.claim_id = cc35.claim_id
                            AND cc35.checklist_item_id = 35
                        INNER JOIN claim_totalrestart AS tr
                            ON c.claim_id = tr.claim_id
                        WHERE c.claim_id = {claimID};'''.format(claimID = tr_cal)

                # cursor.execute(tr_sql_query)
                # save query results as DF
                # tr_temp_df = pd.DataFrame(cursor.fetchall())
                tr_temp_df = pd.DataFrame(mysql_q(vgc_u, vgc_pw, vgc_host, 'visualgap_claims', tr_sql_query, 0, 0))

                # append query result to c_template_df
                tr_consolidate_df = tr_consolidate_df.append(tr_temp_df)

            cal_cols = {0:'claim_nbr', 1:'loss_date', 2:'alt_name', 3:'contact', 4:'first', 5:'last', 6:'gap_payable', 7:'incp_date', 8:'nada_value',
                9:'vehicle', 10:'loss_type', 11:'primary_carrier', 12:'max_benefit', 13:'primary_pymt', 14:'excess_deductible', 15:'term', 16:'totalrestart_payable', 
                17:'scr', 18:'clr', 19:'cdr', 20:'oref', 21:'salvage', 22:'prior_dmg', 23:'other1_amt', 24:'other1_description',
                25:'other2_amt', 26:'other2_description', 27:'other3_amt', 28:'other3_description', 29:'carrier', 30:'subtotal'}

            tr_template_df = tr_consolidate_df.rename(columns = cal_cols)
            tr_template_df = tr_template_df.reset_index(drop=True)

            # Create Calculations
            tr_pdf_template = "letters/pdf_templates/TR_calculation_template.pdf"
            position = 3
            tr_calculations(tr_template_df, tr_pdf_template, position)
                
        # Close SQL Connection
        # cursor.close()
        # cnx.close()

        # create 
        file_list = fileList(file_staging_dir, file_ext)

        # Concatenated output file
        outfn = f'S:/claims/letters/TOTALRESTART_{now.strftime("%Y-%m-%d")}'

        ConCat_pdf(file_list, outfn)

        # Remove files from staging directory
        clear_dir(file_staging_dir, file_ext)

else: print('No amount greater then 0.')

C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_df['payment_amount'] = template_df['payment_amount'].map('${:,.2f}'.format)
C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_df['loss_date'] = pd.to_datetime(template_df['loss_date']).dt.strftime('%B %d, %Y')
C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:7: SettingWithCopyWarning: 
A value is

In [84]:
# Update toVGC to 1
update_tovgc_1(tr_df)

GAP Claims paid via ACH and $0 claims

In [85]:
# GAP Claims paid via ACH and $0 Claims
ach_0_df = gap_pymts_df.loc[(gap_pymts_df['payment_amount'] <= 0) | (gap_pymts_df['pymt_method'].str.contains('ACH'))].copy()
# ach_0_df

In [87]:
# attachment directory
attachment_dir = 'S:/claims/letters/attachment/'

# Remove files from staging directory
clear_dir(file_staging_dir, file_ext) 
clear_dir(attachment_dir, file_ext)

if len(ach_0_df.index) > 0:
    letter_cols = ['claim_nbr', 'loss_date', 'alt_name', 'contact', 'address1', 'city', 'state', 'zip', 'first', 'last', 'acct_number', 'payment_amount', 'StateDesc', 'StateCode', 'f_lang' ]
    temp_cols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]

    # Connect to SQL DB
    # cnx = mc.connect(user=vgc_u, password=vgc_pw,
    #                 host=vgc_host,
    #                 database='visualgap_claims')
    # cursor = cnx.cursor()

    for index, row in ach_0_df.iterrows():
        # Check for GAP or Supplemental
        if row['pymt_type_id'] == 1:
            # GAP
            letter_df = ach_0_df.loc[ach_0_df['claim_id'] == row['claim_id']].copy()       
            template_df = letter_df[letter_cols]
            pdf_template = "letters/pdf_templates/GAP_letter_template.pdf"
            position = 1
            claim_type = 'GAP Claim'
        else:
            # Supplemental
            letter_df = ach_0_df.loc[ach_0_df['claim_id'] == row['claim_id']].copy()       
            template_df = letter_df[letter_cols]
            pdf_template = "letters/pdf_templates/Supp_letter_template.pdf"
            position = 2
            claim_type = 'Supplemental GAP Claim'

        gap_letter(template_df, pdf_template, position)

        # Calculation
        c_consolidate_df = pd.DataFrame(columns = temp_cols)

        c_sql_query = '''
        SELECT c.claim_nbr, c.loss_date, l.alt_name, l.contact, b.first, b.last, cc.gap_payable,
            cl.incp_date, cl.last_payment, cl.interest_rate, cl.amount AS Amt_Fin, cc.balance_last_pay,
            cc.nbr_of_days, cc.per_day, ROUND(cc.payoff,2) AS payoff, (cc.ltv * 100) AS ltv, cc.covered_fin_amount,
            (cc.percent_uncovered * 100) AS percent_uncovered, (cc.ltv_limit * 100) AS ltv_limit, v.nada_value, CONCAT(v.year, ' ', v.make, ' ', v.model) AS vehicle,
            v.deductible, cls.description AS loss_type, cc26.description AS primary_carrier,
            cc14.amount AS past_due, cc15.amount AS late_fees, cc16.amount AS skip_pymts,
            cc17.amount AS skip_fees, cc5.amount AS primary_pymt, cc7.amount AS excess_deductible,
            cc8.amount AS scr, cc9.amount AS clr, cc10.amount AS cdr, cc11.amount AS oref,
            cc18.amount AS salvage, cc19.amount AS prior_dmg, cc20.amount AS over_ltv,
            cc21.amount AS other1_amt, cc21.description AS other1_description,
            cc22.amount AS other2_amt, cc22.description AS other2_description, ca.description AS carrier,
            (cc22.amount + cc21.amount + cc20.amount + cc19.amount + cc18.amount + cc11.amount + cc10.amount + cc9.amount + 
            cc8.amount + cc7.amount + cc5.amount + cc17.amount + cc16.amount + cc15.amount + cc14.amount) as subtotal
        FROM claims c
        INNER JOIN claim_lender l
            USING (claim_id)
        INNER JOIN claim_borrower b
            USING (claim_id)
        INNER JOIN claim_loan cl
            USING (claim_id)
        INNER JOIN claim_calculations cc
            USING (claim_id)
        INNER JOIN claim_vehicle v
            USING (claim_id)
        INNER JOIN claims_loss_type cls
            USING (loss_type_id)
        INNER JOIN carriers ca
            ON c.carrier_id = ca.carrier_id                        
        INNER JOIN claim_checklist AS cc26
            ON c.claim_id = cc26.claim_id
            AND cc26.checklist_item_id = 26    
        INNER JOIN claim_checklist AS cc14
            ON c.claim_id = cc14.claim_id
            AND cc14.checklist_item_id = 14    
        INNER JOIN claim_checklist AS cc15
            ON c.claim_id = cc15.claim_id
            AND cc15.checklist_item_id = 15    
        INNER JOIN claim_checklist AS cc16
            ON c.claim_id = cc16.claim_id
            AND cc16.checklist_item_id = 16       
        INNER JOIN claim_checklist AS cc17
            ON c.claim_id = cc17.claim_id
            AND cc17.checklist_item_id = 17       
        INNER JOIN claim_checklist AS cc5
            ON c.claim_id = cc5.claim_id
            AND cc5.checklist_item_id = 5
        INNER JOIN claim_checklist AS cc7
            ON c.claim_id = cc7.claim_id
            AND cc7.checklist_item_id = 7
        INNER JOIN claim_checklist AS cc8
            ON c.claim_id = cc8.claim_id
            AND cc8.checklist_item_id = 8
        INNER JOIN claim_checklist AS cc9
            ON c.claim_id = cc9.claim_id
            AND cc9.checklist_item_id = 9
        INNER JOIN claim_checklist AS cc10
            ON c.claim_id = cc10.claim_id
            AND cc10.checklist_item_id = 10
        INNER JOIN claim_checklist AS cc11
            ON c.claim_id = cc11.claim_id
            AND cc11.checklist_item_id = 11      
        INNER JOIN claim_checklist AS cc18
            ON c.claim_id = cc18.claim_id
            AND cc18.checklist_item_id = 18
        INNER JOIN claim_checklist AS cc19
            ON c.claim_id = cc19.claim_id
            AND cc19.checklist_item_id = 19
        INNER JOIN claim_checklist AS cc20
            ON c.claim_id = cc20.claim_id
            AND cc20.checklist_item_id = 20     
        INNER JOIN claim_checklist AS cc21
            ON c.claim_id = cc21.claim_id
            AND cc21.checklist_item_id = 21
        INNER JOIN claim_checklist AS cc22
            ON c.claim_id = cc22.claim_id
            AND cc22.checklist_item_id = 22  
        WHERE c.claim_id = {claimID};'''.format(claimID = row['claim_id'])

        # cursor.execute(c_sql_query)
        # # save query results as DF
        # c_temp_df = pd.DataFrame(cursor.fetchall())
        c_temp_df = pd.DataFrame(mysql_q(vgc_u, vgc_pw, vgc_host, 'visualgap_claims', c_sql_query, 0, 0))

        # append query result to c_template_df
        c_consolidate_df = c_consolidate_df.append(c_temp_df)

        cal_cols = {0:'claim_nbr', 1:'loss_date', 2:'alt_name', 3:'contact', 4:'first', 5:'last', 6:'gap_payable', 7:'incp_date', 8:'last_payment', 9:'interest_rate', 10:'Amt_Fin',
            11:'balance_last_pay', 12:'nbr_of_days', 13:'per_day', 14:'payoff', 15:'ltv', 16:'covered_fin_amount', 17:'percent_uncovered', 18:'ltv_limit', 19:'nada_value',
            20:'vehicle', 21:'deductible', 22:'loss_type', 23:'primary_carrier', 24:'past_due', 25:'late_fees', 26:'skip_pymts', 27:'skip_fees', 28:'primary_pymt',
            29:'excess_deductible', 30:'scr', 31:'clr', 32:'cdr', 33:'oref', 34:'salvage', 35:'prior_dmg', 36:'over_ltv', 37:'other1_amt', 38:'other1_description',
            39:'other2_amt', 40:'other2_description', 41:'carrier', 42:'subtotal'}

        c_template_df = c_consolidate_df.rename(columns = cal_cols)
        c_template_df = c_template_df.reset_index(drop=True)

        # Create Calculations
        c_pdf_template = "letters/pdf_templates/GAP_calculation_template.pdf"
        position = 3
        calculations(c_template_df, c_pdf_template, position)

        # create a list of file to concatenate
        file_list = fileList(file_staging_dir, file_ext)

        # Concatenated output file
        outfn = attachment_dir + row['claim_nbr'] + '-' + row['last']
        
        ConCat_pdf(file_list, outfn)

        # send Email
        attach_name = row['claim_nbr'] + '-' + row['last'] + ".pdf"
        missingEmail = 0
        toEmail = []

        if row['email'] == '' and row['email2'] == '':
            toEmail.append('claims@visualgap.com')
            missingEmail = 1
        elif row['email'] == '':
            toEmail.append(row['email2'])
        elif row['email2'] == '':
            toEmail.append(row['email'])
        else:
            toEmail.append(row['email'])
            toEmail.append(row['email2'])

        if missingEmail == 0:
            msg_html = """
            <html>
            <body>
                <p>Hello {}:<br>
                <br>
                We completed the {} for {}'s loan.  Attached is claim letter and calculation. <br>
                <br>
                Thank you, <br>
                Claims Department <br>
                <br>
                <b>Frost Financial Services, Inc. | VisualGAP <br>
                Claims Department <br>
                Phone: 888-753-7678 Option 3</b>
                </p>
            </body>
            </html>
            """.format(row['contact'], claim_type, (row['first'] + ' ' + row['last']))
        else:
            msg_html = """
            <html>
            <body>
                <p>There is not an email address on claim {}:<br>
                <br>
                Please fax or mail the attached to {}. <br>
                <br>
                Thank you, <br>
                Claims Department <br>
                <br>
                <b>Frost Financial Services, Inc. | VisualGAP <br>
                Claims Department <br>
                Phone: 888-753-7678 Option 3</b>
                </p>
            </body>
            </html>
            """.format(row['claim_nbr'], row['alt_name'])

        send_email(toEmail, claim_type, msg_html, attachment_dir, attach_name)

        # Remove files from staging directory
        clear_dir(attachment_dir, file_ext)
        clear_dir(file_staging_dir, file_ext)
        
    # Close SQL Connection
    # cursor.close()
    # cnx.close()        

else: print('No ACH and/or $0 claims.')

C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_df['payment_amount'] = template_df['payment_amount'].map('${:,.2f}'.format)
C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_df['loss_date'] = pd.to_datetime(template_df['loss_date']).dt.strftime('%B %d, %Y')
C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:7: SettingWithCopyWarning: 
A value is

In [88]:
# Update toVGC to 1
update_tovgc_1(ach_0_df)

In [89]:
# Plus Claims paid via ACH and $0 Claims
ach_plus_df = pymts_df.loc[(pymts_df['payment_category_id'] == 2) & ((pymts_df['payment_amount'] <= 0) | (pymts_df['pymt_method'].str.contains('ACH')))].copy()
ach_plus_df

,rtbp_id,claim_id,claim_nbr,carrier_id,alt_name,dealer_securityId,contact,address1,city,state,zip,pymt_method,first,last,pymt_type_id,payment_amount,payment_category_id,check_nbr,batch_id,qb_txnid,acct_number,loss_date,email,email2,expense,checking,VGSecurityId,QB_ListID,carrier,StateId,StateDesc,StateCode,f_lang
9,2317,1475,202110192497,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,OCTAVIO,PARTIDA,1,1000.0,2,0,20220209162206,0,187895-01,2021-08-28,jediracer13@gmail.com,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO,,,,


In [90]:
# Remove files from staging directory
clear_dir(file_staging_dir, file_ext) 
clear_dir(attachment_dir, file_ext)

letter_cols = ['claim_nbr', 'loss_date', 'alt_name', 'contact', 'address1', 'city', 'state', 'zip', 'first', 'last', 'acct_number', 'payment_amount', 'StateDesc', 'StateCode', 'f_lang' ]

if len(ach_plus_df.index) > 0:

    for index, row in ach_plus_df.iterrows():

        # create df for template
        letter_df = ach_plus_df.loc[ach_plus_df['claim_id'] == row['claim_id']].copy()
        template_df = letter_df[letter_cols]
        pdf_template = "letters/pdf_templates/PLUS_letter_template.pdf"
        position = 1
        claim_type = "GAP Plus Claim"

        # Create Plus Letter
        gap_letter(template_df, pdf_template, position)

        s_dir = f"{file_staging_dir}{row['claim_nbr']}-{position}.pdf"
        a_dir = f"{attachment_dir}{row['claim_nbr']}-{row['last']}.pdf"
        # Move and rename file to Attachment directory   
        shutil.move(s_dir, a_dir)

        # send Email
        attach_name = f"{row['claim_nbr']}-{row['last']}.pdf"
        missingEmail = 0
        toEmail = []

        if row['email'] == '' and row['email2'] == '':
            toEmail.append('claims@visualgap.com')
            missingEmail = 1
        elif row['email'] == '':
            toEmail.append(row['email2'])
        elif row['email2'] == '':
            toEmail.append(row['email'])
        else:
            toEmail.append(row['email'])
            toEmail.append(row['email2'])

        if missingEmail == 0:
            msg_html = """
            <html>
            <body>
                <p>Hello {}:<br>
                <br>
                We completed the {} for {}'s loan.  Attached is claim letter and calculation. <br>
                <br>
                Thank you, <br>
                Claims Department <br>
                <br>
                <b>Frost Financial Services, Inc. | VisualGAP <br>
                Claims Department <br>
                Phone: 888-753-7678 Option 3</b>
                </p>
            </body>
            </html>
            """.format(row['contact'], claim_type, (row['first'] + ' ' + row['last']))
        else:
            msg_html = """
            <html>
            <body>
                <p>There is not an email address on claim {}:<br>
                <br>
                Please fax or mail the attached to {}. <br>
                <br>
                Thank you, <br>
                Claims Department <br>
                <br>
                <b>Frost Financial Services, Inc. | VisualGAP <br>
                Claims Department <br>
                Phone: 888-753-7678 Option 3</b>
                </p>
            </body>
            </html>
            """.format(row['claim_nbr'], row['alt_name'])

        send_email(toEmail, claim_type, msg_html, attachment_dir, attach_name)

        # Remove files from staging directory
        clear_dir(attachment_dir, file_ext)

else: print('No ACH Plus.')

C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_df['payment_amount'] = template_df['payment_amount'].map('${:,.2f}'.format)
C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_df['loss_date'] = pd.to_datetime(template_df['loss_date']).dt.strftime('%B %d, %Y')
C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:7: SettingWithCopyWarning: 
A value is

In [91]:
# Update toVGC to 1
update_tovgc_1(ach_plus_df)

In [92]:
# Plus Claims paid via ACH and $0 Claims
tr_ach_0_df = pymts_df.loc[(pymts_df['payment_category_id'] == 3) & ((pymts_df['payment_amount'] <= 0) | (pymts_df['pymt_method'].str.contains('ACH')))].copy()
tr_ach_0_df

,rtbp_id,claim_id,claim_nbr,carrier_id,alt_name,dealer_securityId,contact,address1,city,state,zip,pymt_method,first,last,pymt_type_id,payment_amount,payment_category_id,check_nbr,batch_id,qb_txnid,acct_number,loss_date,email,email2,expense,checking,VGSecurityId,QB_ListID,carrier,StateId,StateDesc,StateCode,f_lang
16,2319,1471,202110182493,12,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,Check,ANDREW,FLYNN,1,0.0,3,0,20220209162206,0,0,2021-09-10,jediracer13@gmail.com,,Claims Paid:TotalRestart,TotalRestart Claims,46724,80000006-1631298724,Fortega,,,,


In [93]:
# Remove files from staging directory
clear_dir(file_staging_dir, file_ext) 
clear_dir(attachment_dir, file_ext)

if len(tr_ach_0_df.index) > 0:
    letter_cols = ['claim_nbr', 'loss_date', 'alt_name', 'contact', 'address1', 'city', 'state', 'zip', 'first', 'last', 'acct_number', 'payment_amount', 'StateDesc', 'StateCode', 'f_lang' ]
    temp_cols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]

    # Connect to SQL DB
    # cnx = mc.connect(user=vgc_u, password=vgc_pw,
    #                 host=vgc_host,
    #                 database='visualgap_claims')
    # cursor = cnx.cursor()

    for index, row in tr_ach_0_df.iterrows():
        # Check for GAP or Supplemental
        if row['pymt_type_id'] == 1:
            # GAP
            letter_df = tr_ach_0_df.loc[tr_ach_0_df['claim_id'] == row['claim_id']].copy()       
            template_df = letter_df[letter_cols]
            pdf_template = "letters/pdf_templates/TR_letter_template.pdf"
            position = 1
            claim_type = 'TotalRestart Claim'
        else:
            # Supplemental
            letter_df = ach_0_df.loc[ach_0_df['claim_id'] == row['claim_id']].copy()       
            template_df = letter_df[letter_cols]
            pdf_template = "letters/pdf_templates/TR_letter_template.pdf"
            position = 2
            claim_type = 'Supplemental TotalRestart Claim'

        gap_letter(template_df, pdf_template, position)

        # Calculation
        c_consolidate_df = pd.DataFrame(columns = temp_cols)

        c_sql_query = '''
                SELECT c.claim_nbr, c.loss_date, l.alt_name, l.contact, b.first, b.last, cc.gap_payable,
                    cl.incp_date, v.nada_value, CONCAT(v.year, ' ', v.make, ' ', v.model) AS vehicle,
                    cls.description AS loss_type, cc26.description AS primary_carrier, tr.max_benefit,
                    cc5.amount AS primary_pymt, cc7.amount AS excess_deductible, tr.term, tr.totalrestart_payable,
                    cc8.amount AS scr, cc9.amount AS clr, cc10.amount AS cdr, cc11.amount AS oref,
                    cc18.amount AS salvage, cc19.amount AS prior_dmg, 
                    cc21.amount AS other1_amt, cc21.description AS other1_description,
                    cc22.amount AS other2_amt, cc22.description AS other2_description, 
                    cc35.amount AS other3_amt, cc35.description AS other3_description, ca.description AS carrier,
                    (cc5.amount + cc7.amount + cc18.amount + cc19.amount + cc.gap_payable + cc8.amount + cc8.amount + 
                    cc9.amount + cc10.amount + cc11.amount + cc21.amount + cc22.amount + cc35.amount) as subtotal
                FROM claims c
                INNER JOIN claim_lender l
                    USING (claim_id)
                INNER JOIN claim_borrower b
                    USING (claim_id)
                INNER JOIN claim_loan cl
                    USING (claim_id)
                INNER JOIN claim_calculations cc
                    USING (claim_id)
                INNER JOIN claim_vehicle v
                    USING (claim_id)
                INNER JOIN claims_loss_type cls
                    USING (loss_type_id)
                INNER JOIN carriers ca
                    ON c.carrier_id = ca.carrier_id                        
                INNER JOIN claim_checklist AS cc26
                    ON c.claim_id = cc26.claim_id
                    AND cc26.checklist_item_id = 26                  
                INNER JOIN claim_checklist AS cc5
                    ON c.claim_id = cc5.claim_id
                    AND cc5.checklist_item_id = 5
                INNER JOIN claim_checklist AS cc7
                    ON c.claim_id = cc7.claim_id
                    AND cc7.checklist_item_id = 7
                INNER JOIN claim_checklist AS cc8
                    ON c.claim_id = cc8.claim_id
                    AND cc8.checklist_item_id = 8
                INNER JOIN claim_checklist AS cc9
                    ON c.claim_id = cc9.claim_id
                    AND cc9.checklist_item_id = 9
                INNER JOIN claim_checklist AS cc10
                    ON c.claim_id = cc10.claim_id
                    AND cc10.checklist_item_id = 10
                INNER JOIN claim_checklist AS cc11
                    ON c.claim_id = cc11.claim_id
                    AND cc11.checklist_item_id = 11      
                INNER JOIN claim_checklist AS cc18
                    ON c.claim_id = cc18.claim_id
                    AND cc18.checklist_item_id = 18
                INNER JOIN claim_checklist AS cc19
                    ON c.claim_id = cc19.claim_id
                    AND cc19.checklist_item_id = 19     
                INNER JOIN claim_checklist AS cc21
                    ON c.claim_id = cc21.claim_id
                    AND cc21.checklist_item_id = 21
                INNER JOIN claim_checklist AS cc22
                    ON c.claim_id = cc22.claim_id
                    AND cc22.checklist_item_id = 22
                INNER JOIN claim_checklist AS cc35
                    ON c.claim_id = cc35.claim_id
                    AND cc35.checklist_item_id = 35
                INNER JOIN claim_totalrestart AS tr
                    ON c.claim_id = tr.claim_id
                WHERE c.claim_id = {claimID};'''.format(claimID = row['claim_id'])

        # cursor.execute(c_sql_query)
        # # save query results as DF
        # c_temp_df = pd.DataFrame(cursor.fetchall())
        c_temp_df = pd.DataFrame(mysql_q(vgc_u, vgc_pw, vgc_host, 'visualgap_claims', c_sql_query, 0, 0))

        # append query result to c_template_df
        c_consolidate_df = c_consolidate_df.append(c_temp_df)

        cal_cols = {0:'claim_nbr', 1:'loss_date', 2:'alt_name', 3:'contact', 4:'first', 5:'last', 6:'gap_payable', 7:'incp_date', 8:'nada_value',
            9:'vehicle', 10:'loss_type', 11:'primary_carrier', 12:'max_benefit', 13:'primary_pymt', 14:'excess_deductible', 15:'term', 16:'totalrestart_payable', 
            17:'scr', 18:'clr', 19:'cdr', 20:'oref', 21:'salvage', 22:'prior_dmg', 23:'other1_amt', 24:'other1_description',
            25:'other2_amt', 26:'other2_description', 27:'other3_amt', 28:'other3_description', 29:'carrier', 30:'subtotal'}

        c_template_df = c_consolidate_df.rename(columns = cal_cols)
        c_template_df = c_template_df.reset_index(drop=True)

        # Create Calculations
        c_pdf_template = "letters/pdf_templates/TR_calculation_template.pdf"
        position = 3
        tr_calculations(c_template_df, c_pdf_template, position)

        # create a list of file to concatenate
        file_list = fileList(file_staging_dir, file_ext)

        # Concatenated output file
        outfn = attachment_dir + row['claim_nbr'] + '-' + row['last']
        
        ConCat_pdf(file_list, outfn)

        # send Email
        attach_name = row['claim_nbr'] + '-' + row['last'] + ".pdf"
        missingEmail = 0
        toEmail = []

        if row['email'] == '' and row['email2'] == '':
            toEmail.append('claims@visualgap.com')
            missingEmail = 1
        elif row['email'] == '':
            toEmail.append(row['email2'])
        elif row['email2'] == '':
            toEmail.append(row['email'])
        else:
            toEmail.append(row['email'])
            toEmail.append(row['email2'])

        if missingEmail == 0:
            msg_html = """
            <html>
            <body>
                <p>Hello {}:<br>
                <br>
                We completed the {} for {}'s loan.  Attached is claim letter and calculation. <br>
                <br>
                Thank you, <br>
                Claims Department <br>
                <br>
                <b>Frost Financial Services, Inc. | VisualGAP <br>
                Claims Department <br>
                Phone: 888-753-7678 Option 3</b>
                </p>
            </body>
            </html>
            """.format(row['contact'], claim_type, (row['first'] + ' ' + row['last']))
        else:
            msg_html = """
            <html>
            <body>
                <p>There is not an email address on claim {}:<br>
                <br>
                Please fax or mail the attached to {}. <br>
                <br>
                Thank you, <br>
                Claims Department <br>
                <br>
                <b>Frost Financial Services, Inc. | VisualGAP <br>
                Claims Department <br>
                Phone: 888-753-7678 Option 3</b>
                </p>
            </body>
            </html>
            """.format(row['claim_nbr'], row['alt_name'])

        send_email(toEmail, claim_type, msg_html, attachment_dir, attach_name)

        # Remove files from staging directory
        clear_dir(attachment_dir, file_ext)
        clear_dir(file_staging_dir, file_ext)
        
    # Close SQL Connection
    # cursor.close()
    # cnx.close()        

else: print('No ACH and/or $0 claims.')

C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_df['payment_amount'] = template_df['payment_amount'].map('${:,.2f}'.format)
C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_df['loss_date'] = pd.to_datetime(template_df['loss_date']).dt.strftime('%B %d, %Y')
C:\Users\jbehler\AppData\Local\Temp/ipykernel_29760/2586907294.py:7: SettingWithCopyWarning: 
A value is

In [94]:
# Update toVGC to 1
update_tovgc_1(tr_ach_0_df)

FINAL REPORT

In [95]:
# Create Final Report
attach_file = f'{attachment_dir}Claims_Paid_{now.strftime("%Y-%m-%d")}.pdf'
# html_file = f'{attachment_dir}final_rpt.html'
html_file = f'{file_staging_dir}final_rpt.html'
csv_file = f'{file_staging_dir}final_rpt.csv'
# template_file = f'{attachment_dir}templates/pymt_summary_template.html'
template_file = './html/pymt_summary_template.html'
# html_file2 = f'{attachment_dir}Claims_Paid.html'
html_file2 = './letters/staging/Claims_Paid.html'
# using pymts_df for development
# Collect needed fields
final_rpt_df = pymts_df[['claim_nbr', 'carrier', 'alt_name', 'first', 'last', 'payment_amount', 'pymt_method', 'check_nbr', 'pymt_type_id', 'payment_category_id']].copy()

In [96]:
# format df
# list of conditions
type_conds = [((final_rpt_df['payment_category_id'] == 1) & (final_rpt_df['pymt_type_id'] == 1)),
              ((final_rpt_df['payment_category_id'] == 1) & (final_rpt_df['pymt_type_id'] == 2)),
              ((final_rpt_df['payment_category_id'] == 2) & (final_rpt_df['pymt_type_id'] == 1)),
              ((final_rpt_df['payment_category_id'] == 2) & (final_rpt_df['pymt_type_id'] == 2)),
              ((final_rpt_df['payment_category_id'] == 3) & (final_rpt_df['pymt_type_id'] == 1)),
              ((final_rpt_df['payment_category_id'] == 3) & (final_rpt_df['pymt_type_id'] == 2)),]
# list of name types
type_name = ['GAP', 'GAP Supp', 'GAP Plus', 'GAP Plus Supp', 'TotalRestart', 'TotalRestart Supp']

# add column and assigne values
final_rpt_df['Claim_Type'] = np.select(type_conds, type_name)
# final_rpt_df

In [97]:
# remove pymt_type_id and Payment_category_id columns
final_rpt_df.drop(['pymt_type_id', 'payment_category_id'], axis=1, inplace=True)
final_rpt_df.rename(columns = {'claim_nbr':'Claim Nbr', 'carrier':'Carrier', 'alt_name':'Lender', 'first':'First Name', 'last':'Last Name', 'payment_amount':'Amount', 'pymt_method':'Method',
                               'check_nbr':'Check Nbr', 'Claim_Type':'Claim Type'}, inplace=True)
# format df
# df_format = {'Amount':'${0:,.2f}'}
# final_rpt_df = final_rpt_df.style.format(df_format)
final_rpt_df['Amount'] = final_rpt_df['Amount'].map('${:,.2f}'.format)

final_rpt_df

,Claim Nbr,Carrier,Lender,First Name,Last Name,Amount,Method,Check Nbr,Claim Type
0,202109172488,ANICO,Charlotte Metro Credit Union,Sam,Hunt,"$5,302.64",Check,0,GAP
1,202110182493,ANICO,Charlotte Metro Credit Union,ANDREW,FLYNN,$0.00,ACH-GL,0,GAP
2,202110182494,ANICO,Charlotte Metro Credit Union,TYSHAUN,GRANT,"$5,138.04",ACH-GL,0,GAP
3,202110192496,ANICO,City Credit Union,JOSHUA,WELCH,$0.00,Check,0,GAP
4,202110192497,ANICO,Charlotte Metro Credit Union,OCTAVIO,PARTIDA,$653.87,ACH-GL,0,GAP
5,202110192498,ANICO,Charlotte Metro Credit Union,SHERRY,VOGT,$0.00,ACH-GL,0,GAP
6,202110192499,ANICO,City Credit Union,BRIAN,CHANDLER,$191.17,Check,0,GAP Supp
7,202110272510,ANICO,City Credit Union,TYSHAUN,GRANT,"$3,498.00",Check,0,GAP
8,202111092512,ANICO,Charlotte Metro Credit Union,Ralph,Sutton,$0.00,ACH-GL,0,GAP
9,202110192497,ANICO,Charlotte Metro Credit Union,OCTAVIO,PARTIDA,"$1,000.00",ACH-GL,0,GAP Plus


In [98]:
# export is df as csv
final_rpt_df.to_csv(csv_file, index=False)
# read in csv
csvFile = pd.read_csv(csv_file)
# convert csv to html
csvFile.to_html(html_file, index=False)

In [99]:
# Get html df
soup = Soup(open(html_file), "html.parser")
table = str(soup.select_one("table", {"class":"dataframe"}))
# Get template
soup2 = Soup(open(template_file), "html.parser")
df_div = soup2.find("div", {"id":"df"})
# append table to template
df_div.append(Soup(table, 'html.parser'))
# write html file
with open(html_file2,'w') as file:
    file.write(str(soup2))

In [100]:
# create PDF from html
pdf_options = {'orientation': 'landscape',
                'page-size': 'Letter',
                'margin-top': '0.25in',
                'margin-right': '0.25in',
                'margin-bottom': '0.25in',
                'margin-left': '0.25in',
                'encoding': "UTF-8",}
pdfkit.from_file(html_file2, attach_file, options=pdf_options)

True